In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors
import csv
import numpy as np
import pandas as pd
import cobra

In [2]:
cobra.__version__

'0.15.4'

In [3]:
Annotation = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Annotation', index_col=0)
Annotation.index = Annotation.index.map(str)
Annotation = Annotation.fillna('')
Transcriptomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Transcriptomics', header=[0,1,2,3], index_col=0)
Transcriptomics.index = Transcriptomics.index.map(str)
Proteomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Proteomics', header=[0,1,2], index_col=0)
Proteomics.index = Proteomics.index.map(str)
Fitness = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Fitness', index_col=0)
Fitness.index = Fitness.index.map(str)

In [4]:
def background_gradient(s, cmap='seismic', text_color_threshold=0.408):
    lim = max(abs(s.min().min()),abs(s.max().max()))
    rng = 2.0*lim
    norm = colors.Normalize(-lim - (rng * 0.2), lim + (rng * 0.2))
    rgbas = plt.cm.get_cmap(cmap)(norm(s.values))
    def relative_luminance(rgba):
        r, g, b = (x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055 ** 2.4) for x in rgba[:3])
        return 0.2126 * r + 0.7152 * g + 0.0722 * b
    def css(rgba):
        dark = relative_luminance(rgba) < text_color_threshold
        text_color = '#f1f1f1' if dark else '#000000'
        return 'background-color: {b};color: {c};'.format(b=colors.rgb2hex(rgba), c=text_color)

    if s.ndim == 1:
        return [css(rgba) for rgba in rgbas]
    else:
        return pd.DataFrame([[css(rgba) for rgba in row] for row in rgbas], index=s.index, columns=s.columns)

def Show_Data(x):
    display(Transcriptomics.loc[x].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Proteomics.index]
    display(Proteomics.loc[temp].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Fitness.index]
    display(Fitness.loc[temp].style.apply(background_gradient, cmap='seismic', axis=None))
    return;

In [5]:
eco = cobra.io.load_json_model('../../Data/BiGG_Models/iML1515.json')
sce = cobra.io.load_json_model('../../Data/BiGG_Models/iMM904.json')
hsa = cobra.io.load_json_model('../../Data/BiGG_Models/RECON1.json')
hsa2 = cobra.io.load_json_model('../../Data/BiGG_Models/Recon3D.json')
ptri = cobra.io.load_json_model('../../Data/BiGG_Models/iLB1027_lipid.json')
cre = cobra.io.load_json_model('../../Data/BiGG_Models/iRC1080.json')

In [6]:
model = cobra.io.load_json_model("IFO0880_GPR_1a.json")

In [7]:
print(len(model.genes))
print(len([x for x in model.genes if not x.id[0].isalpha()]))
model

1362
1149


Name,R. toruloides
Memory address,0x01027435ac8
Number of metabolites,3409
Number of reactions,3510
Number of groups,0
Objective expression,0
Compartments,"c, x, m, e, r, v, n, g, p, h, s, f, l"


### Duplicate reactions

In [8]:
duplicated = set()
for i, r in enumerate(model.reactions):
    temp = 0
    for j, r2 in enumerate(model.reactions):
        if j > i and r2.id not in duplicated:
            if r.reactants == r2.reactants and r.products == r2.products:
                if temp == 0:
                    temp = 1
                    duplicated.add(r.id)
                    print(r.id, r.reaction, r.gene_reaction_rule)
                duplicated.add(r2.id)
                print(r2.id, r2.reaction, r2.gene_reaction_rule)
    if temp == 1:
        print()

ARD dhmtp_c + o2_c --> 2kmb_c + for_c + 2.0 h_c 16330
ACDO dhmtp_c + o2_c --> 2kmb_c + for_c + h_c 16330

yli_R1488 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1487 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1489 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1490 atp_c + btn_c --> btamp_c + ppi_c 16404

yli_R0002 akg_c + gln__L_c + h_c + nadph_c --> glu__L_c + nadp_c 15713
GLUSy akg_c + gln__L_c + h_c + nadph_c --> 2.0 glu__L_c + nadp_c (PP_5075 and 15713) or (b3213 and 15713)

yli_R1425 co2_x + mlthf_x + nadh_x + nh4_x --> gly_x + nad_x + thf_x 12898
yli_R1378 co2_x + mlthf_x + nadh_x + nh4_x --> gly_x + nad_x + thf_x 10040
yli_R1377 co2_x + mlthf_x + nadh_x + nh4_x --> gly_x + nad_x + thf_x 10205

yli_R1466 2ippm_m + h2o_m <=> 3c2hmp_m 14914
yli_R7859 2ippm_m + h2o_m <=> 3c2hmp_m 14914

yli_R8859 3c3hmp_m <=> 2ippm_m + h2o_m 14914
yli_R1465 3c3hmp_m <=> 2ippm_m + h2o_m 14914

CERH124_copy2 cer1_24_c + h_c + nadph_c + o2_c --> cer2_24_c + h2o_c + nadp_c 9664
CERH124_copy1 cer1_24_c 

ANNATn atp_n + 2.0 h_n + nmn_n <=> nad_n + ppi_n 10430
NMNATn atp_n + h_n + nmn_n --> nad_n + ppi_n 10430

CLHCOtex cl_e + 2.0 hco3_c --> cl_c + 2.0 hco3_e 14119 or 15736
CLHCO3tex2 2.0 cl_e + hco3_c --> 2.0 cl_c + hco3_e 16202

FACOAL204_copy1 arachd_c + atp_c + coa_c <=> amp_c + arachdcoa_c + ppi_c 12538 or 12555
FACOAL204_copy2 arachd_c + atp_c + coa_c --> amp_c + arachdcoa_c + ppi_c 11167 or 12538 or 12555 or 15748



#### ARD

In [9]:
for r in sorted(model.genes.get_by_id('16330').reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)

ACDO dhmtp_c + o2_c --> 2kmb_c + for_c + h_c 16330
ARD dhmtp_c + o2_c --> 2kmb_c + for_c + 2.0 h_c 16330
ARD1 dhmtp_c + o2_c --> co_c + for_c + h_c + mtpp_c 16330
DKMPPD2 dkmpp_c + 3.0 h2o_c --> 2kmb_c + for_c + 6.0 h_c + pi_c YEL038W and 16330
yli_R1495 dhmtp_c + o2_c --> co_c + for_c + mtpp_c 16330


In [10]:
# 5mta -> 5mdr1p -> 5mdru1p -> dkmpp
for r in sorted(model.metabolites.get_by_id('5mta_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('5mdru1p_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dkmpp_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dhmtp_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('2kmb_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('mtpp_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACPCS amet_c --> 1acpc_c + 5mta_c + h_c 8613
ACPT16018111Z 12dgr16018111Z_c + amet_c --> 5mta_c + dghs16018111Z_c + h_c 10576
ACPT1601819Z 12dgr1601819Z_c + amet_c --> 5mta_c + dghs1601819Z_c + h_c 10576
ACPT18111Z18111Z 12dgr18111Z18111Z_c + amet_c --> 5mta_c + dghs18111Z18111Z_c + h_c 10576
ACPT18111Z1819Z 12dgr18111Z1819Z_c + amet_c --> 5mta_c + dghs18111Z1819Z_c + h_c 10576
ACPT1819Z18111Z 12dgr1819Z18111Z_c + amet_c --> 5mta_c + dghs1819Z18111Z_c + h_c 10576
ACPT1819Z1819Z 12dgr1819Z1819Z_c + amet_c --> 5mta_c + dghs1819Z1819Z_c + h_c 10576
MTAP 5mta_c + pi_c --> 5mdr1p_c + ade_c 14521 or 8372
SPMS ametam_c + ptrc_c --> 5mta_c + h_c + spmd_c 10577 or 16833
SPRMS ametam_c + spmd_c --> 5mta_c + h_c + sprm_c 10577 or 16833

MDRPD 5mdru1p_c --> dkmpp_c + h2o_c 15829
MTRI 5mdr1p_c <=> 5mdru1p_c 13385 or 15595

DKMPPD2 dkmpp_c + 3.0 h2o_c --> 2kmb_c + for_c + 6.0 h_c + pi_c YEL038W and 16330
MDRPD 5mdru1p_c --> dkmpp_c + h2o_c 15829

ACDO dhmtp_c + o2_c --> 2kmb_c + for_c + h_c 16330
AR

In [11]:
temp = ['8372','14521','13385','15595','15829','11455','16330','12407','14281','14908','15839','8936']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8372,"K00772: mtaP, MTAP; 5'-methylthioadenosine pho...",,MEU1,MTAP,MEU1,MTAP,Not Essential,"cysk 19, cyto 7",GRQ*
14521,"K00757: udp, UPP; uridine phosphorylase",,,,,,Not Essential,"extr 11, mito 6, cyto 6, cyto_mito 6",RGE*
13385,K08963: mtnA; methylthioribose-1-phosphate iso...,,MRI1,MRI1,MRI1,MRI1,Not Essential,"cyto 20, cysk 4, mito 2",YDLR
15595,K03239: EIF2B1; translation initiation factor ...,,GCN3,EIF2B1,GCN3,EIF2B,Not Essential,"mito 8, cyto 7, cyto_nucl 5.5, extr 4, nucl 2,...",YAE*
15829,K08964: mtnB; methylthioribulose-1-phosphate d...,,MDE1,APIP,MDE1,APIP,Not Essential,"pero 7.5, cyto_pero 6.5, cysk 6, mito 5, cyto ...",TKQ*
11455,HMMPfam:haloacid dehalogenase-like hydrolase:P...,,YNL010W,,YNL010W,,Not Essential,"cyto 19.5, cyto_nucl 12.333, cyto_mito 11.666,...",AKK*
16330,"K08967: mtnD, mtnZ, ADI1; 1,2-dihydroxy-3-keto...",,ADI1,ADI1,ADI1,ADI1,Not Essential,"nucl 13.5, cyto_nucl 12.5, cyto 8.5, pero 3",IKA*
12407,"K00825: AADAT, KAT2; kynurenine/2-aminoadipate...",,ARO8,AADAT,ARO8,AADAT,Not Essential,"cyto 11.5, cyto_nucl 11.5, nucl 8.5, pero 6",EKA*
14281,"K14455: GOT2; aspartate aminotransferase, mito...",,AAT1,GOT2,AAT2,GOT2,Not Essential,mito 26,NDA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8372,-0.268657,-0.234283,0.00572781,0.494423,-0.527027,-0.116889,-0.46519,-1.91569,0.170461,-0.556624,-0.682021
14521,-0.00706592,0.25772,0.022661,0.24152,0.653536,0.454526,0.31852,-0.151502,-0.000959473,0.252956,0.0155481
13385,-0.183169,0.131527,-0.0895433,0.189403,0.740175,-0.27821,0.517699,-0.922633,-0.283394,0.265367,1.42844
15595,0.248649,0.278266,-0.180005,0.217383,-0.814564,-0.133952,-0.679194,-0.553147,0.591779,0.279251,0.00149673
15829,-0.0422372,-0.103604,-0.219452,-0.227858,1.56949,-0.186447,0.945233,0.946233,0.0554741,0.471732,-0.282393
11455,0.387876,0.457422,0.271382,0.32656,0.17532,0.0348011,0.0796446,0.0216117,0.206991,0.228272,-0.227832
16330,0.120232,0.474802,-0.0189183,-0.0584658,0.317082,0.258953,0.116167,-0.135116,0.167846,0.258171,0.412204
12407,-0.120884,0.0459414,-0.0417789,-0.263358,-0.162155,-0.400014,0.275532,0.206367,0.0858757,0.335526,1.09564
14281,-0.134443,0.0025888,-0.314789,-0.228214,-0.0206698,-0.461234,-0.785788,-2.1105,-0.415618,-0.136043,-0.195287


S-methyl-5'-thioadenosine degradation  
MTAP 5mta_c + pi_c --> 5mdr1p_c + ade_c 14521 or 8372  
8372 cysk MEU1  
14521 extr uridine phosphorylase (also catalyze deoxy) -> not MTAP

MTRI 5mdr1p_c <=> 5mdru1p_c 13385 or 15595  
13385 cyto mtnA/MRI1  
15595 cito GCN3 translation initiation factor eIF-2B subunit alpha EIF2B1 -> not MTRI

MDRPD 5mdru1p_c --> dkmpp_c + h2o_c 15829  
15829 cyto_pero mtnB/MDE1

DKMPPD2 dkmpp_c + 3.0 h2o_c --> 2kmb_c + for_c + 6.0 h_c + pi_c YEL038W and 16330  
ACRS, 2OH3K5MPPISO, ACDO lumped, incorrect stoichiometry

DKMPPD3_1 - dkmpp + h2o -> dhmtp + pi + h by mtnC (ACRS, 2OH3K5MPPISO lumped)  
ACRS - dkmpp -> hkmpp + h by mtnW/mtnC  
2OH3K5MPPISO - hkmpp + h2o -> dhmtp + pi by mtnX/mtnC  
11455 cyto mtnX blast to 11455

ACDO dhmtp_c + o2_c --> 2kmb_c + for_c + h_c 16330, fe2+ requiring  
ARD dhmtp_c + o2_c --> 2kmb_c + for_c + 2.0 h_c 16330, incorrect stoichiometry  
ARD1 dhmtp_c + o2_c --> co_c + for_c + h_c + mtpp_c 16330, ni2+ requiring  
16330 cyto_nucl mtnD/ADI1 is fe2+ requiring and produce 2kmb and for -> keep ACDO

UNK3 2kmb_c + glu__L_c --> akg_c + met__L_c 12407 or 14281 or 14908 or 15839 or 8936  
UNK3 reaction is different in the pathway page, it is two step.  
2kmb + gln_L -> 2ogm + met_L by mtnE  
2ogm + h2o -> akg + nh4 by mtnU  
The enzyme page 2.6.1.57 is consistent with UNK3, but 2.6.1.57 is not this reaction.  
Blast of known enzymes results in mixed results  
KEGG says this reaction is possible by 2.6.1.5 or 2.6.1.57  
12407 cyto_nucl ARO8 tryptophan aminotransferase 2.6.1.27  
14281 mito AAT1/AAT2 aspartate aminotransferase, mitochondrial 2.6.1.1  
14908 cyto ARO8 aromatic amino acid aminotransferase I / 2-aminoadipate transaminase 2.6.1.57/2.6.1.39/2.6.1.27/2.6.1.5  
15839 cysk ARO8 aromatic amino acid aminotransferase I / 2-aminoadipate transaminase 2.6.1.57/2.6.1.39/2.6.1.27/2.6.1.5  
8936 cyto AAT2 aspartate aminotransferase, cytoplasmic 2.6.1.1  
Keep genes for now

Change MTAP genes to '8372'  
Change MTRI genes to '13385'  
Add ACRS and 2OH3K5MPPISO, change genes '11455'  
Remove DKMPPD2, ARD, ARD1, yli_R1495

In [12]:
model.reactions.get_by_id('MTAP').gene_reaction_rule = '8372'
model.reactions.get_by_id('MTRI').gene_reaction_rule = '13385'
r1 = cre.reactions.get_by_id('ACRS').copy()
r1.gene_reaction_rule = '11455'
r2 = cre.reactions.get_by_id('2OH3K5MPPISO').copy()
r2.gene_reaction_rule = '11455'
model.add_reactions([r1,r2])
model.remove_reactions(['DKMPPD2','ARD','ARD1','yli_R1495'],remove_orphans=True)

In [13]:
for r in sorted(model.genes.get_by_id('8372').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13385').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15829').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11455').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16330').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

MTAP 5mta_c + pi_c --> 5mdr1p_c + ade_c 8372

MTRI 5mdr1p_c <=> 5mdru1p_c 13385
MTRK 5mtr_c + atp_c --> 5mdr1p_c + adp_c + h_c 13385

MDRPD 5mdru1p_c --> dkmpp_c + h2o_c 15829

2OH3K5MPPISO h2o_c + hkmpp_c --> dhmtp_c + pi_c 11455
ACRS dkmpp_c --> h_c + hkmpp_c 11455

ACDO dhmtp_c + o2_c --> 2kmb_c + for_c + h_c 16330


In [14]:
for r in sorted(model.metabolites.get_by_id('5mtr_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('5mdr1p_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

MTRK 5mtr_c + atp_c --> 5mdr1p_c + adp_c + h_c 13385

MTAP 5mta_c + pi_c --> 5mdr1p_c + ade_c 8372
MTRI 5mdr1p_c <=> 5mdru1p_c 13385
MTRK 5mtr_c + atp_c --> 5mdr1p_c + adp_c + h_c 13385


MTRK is catalyzed by mtnK/MTK1  
Blast of uniprot reviewed mtnK seqs results in no hit

In [15]:
model.reactions.get_by_id('MTRK').remove_from_model(remove_orphans=True)

#### btn

In [16]:
for r in sorted(model.genes.get_by_id('16404').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

BACCL atp_c + btn_c + h_c --> btamp_c + ppi_c 16404
BACCLm atp_m + btn_m + h_m --> btamp_m + ppi_m 16404
BTNPL apoC_Lys_c + btamp_c --> amp_c + apoC_Lys_btn_c + h_c 16404
BTNPLm apoC_Lys_m + btamp_m --> amp_m + apoC_Lys_btn_m + h_m 16404
yli_R1487 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1488 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1489 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1490 atp_c + btn_c --> btamp_c + ppi_c 16404


In [17]:
for r in sorted(model.metabolites.get_by_id('btn_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('btamp_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('apoC_Lys_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('apoC_Lys_btn_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

BACCL atp_c + btn_c + h_c --> btamp_c + ppi_c 16404
BTS5 2fe2s_c + amet_c + dtbt_c --> 2fe1s_c + btn_c + dad_5_c + h_c + met__L_c 15908
BTSr dtbt_c + s_c <=> btn_c + 2.0 h_c 15908
yli_R1487 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1488 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1489 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1490 atp_c + btn_c --> btamp_c + ppi_c 16404

BACCL atp_c + btn_c + h_c --> btamp_c + ppi_c 16404
BTNPL apoC_Lys_c + btamp_c --> amp_c + apoC_Lys_btn_c + h_c 16404
yli_R1487 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1488 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1489 atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1490 atp_c + btn_c --> btamp_c + ppi_c 16404

BTNPL apoC_Lys_c + btamp_c --> amp_c + apoC_Lys_btn_c + h_c 16404

BTNPL apoC_Lys_c + btamp_c --> amp_c + apoC_Lys_btn_c + h_c 16404


In [18]:
for r in sorted(model.metabolites.get_by_id('btn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('btamp_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('apoC_Lys_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('apoC_Lys_btn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

BACCLm atp_m + btn_m + h_m --> btamp_m + ppi_m 16404

BACCLm atp_m + btn_m + h_m --> btamp_m + ppi_m 16404
BTNPLm apoC_Lys_m + btamp_m --> amp_m + apoC_Lys_btn_m + h_m 16404

BTNPLm apoC_Lys_m + btamp_m --> amp_m + apoC_Lys_btn_m + h_m 16404

BTNPLm apoC_Lys_m + btamp_m --> amp_m + apoC_Lys_btn_m + h_m 16404


In [19]:
temp = ['12736','12731','15908','16404']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12736,K00652: bioF; 8-amino-7-oxononanoate synthase,,,,LCB2,GCAT,Essential,"mito 22, cyto 3",AKL*
12731,K19562: BIO3-BIO1; bifunctional dethiobiotin s...,,,,BIO3,,Essential,"mito 11, extr 10, cyto 3, pero 2",GAS*
15908,K01012: bioB; biotin synthase,,BIO2,,BIO2,,Essential,"mito 26.5, cyto_mito 14",VAA*
16404,K01942: HLCS; biotin--protein ligase,,BPL1,HLCS,BPL1,HLCS,Essential,"extr 12, cyto 6, cyto_nucl 5.5, mito 4, nucl 3",KSG*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16404,0.224466,-0.772008,-0.234875,-0.5554,-0.0770478,0.701184,0.14778,-0.208173,-1.19924,-0.980346,0.058461


In [20]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

AOXSr2: ala__L_c + pimACP_c --> 8aonn_c + ACP_c + co2_c 12736

AMAOTr: 8aonn_c + amet_c <=> amob_c + dann_c 12731

BTS5: 2fe2s_c + amet_c + dtbt_c --> 2fe1s_c + btn_c + dad_5_c + h_c + met__L_c 15908
BTSr: dtbt_c + s_c <=> btn_c + 2.0 h_c 15908

BACCL: atp_c + btn_c + h_c --> btamp_c + ppi_c 16404
BACCLm: atp_m + btn_m + h_m --> btamp_m + ppi_m 16404
BTNPL: apoC_Lys_c + btamp_c --> amp_c + apoC_Lys_btn_c + h_c 16404
BTNPLm: apoC_Lys_m + btamp_m --> amp_m + apoC_Lys_btn_m + h_m 16404
yli_R1487: atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1488: atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1489: atp_c + btn_c --> btamp_c + ppi_c 16404
yli_R1490: atp_c + btn_c --> btamp_c + ppi_c 16404



Biotin  
16404 extr 12 cyto 6 cyto_nucl 5.5 mito 4 BPL1 biotin---protein ligase  
required for acetyl-CoA carboxylase (ACC1, cyto), should it be part of that reaction?  
BACCL has correct stoichiomtery

bioF is peroxisomal http://www.jbc.org/content/286/35/30455  
pimACP or pimcoa works as a substrate for 8aonn synthesis  
It is not clear how pimelate or pimcoa is synthesized  
pimeloyl-CoA biosynthesis in peroxisome? beta-oxidation? http://www.jbc.org/content/286/49/42133  
Check genes near 12731-12736, maybe a cluster  
In E. coli, it is fatty acid synthesis https://www.nature.com/articles/nchembio.420  
In B. subtilis, it is from long chain acyl-acp by C450 enzyme https://www.ncbi.nlm.nih.gov/pubmed/11368323  
The rest of the biotin biosynthesis is mitochondrial  
15908 mito BIO2 biotin synthase -> check 2Fe-2S ferredoxin metabolites  
BTS5 has correct stoichiomtery

Remove BTSr, yli_R1487, yli_R1488, yli_R1489, yli_R1490, BACCLm, BTNPLm

In [21]:
m = model.metabolites.get_by_id('pimACP_c')
m.id = 'pimcoa_c'
m.name = 'Pimeloyl-CoA'
m.formula = 'C28H41N7O19P3S'
m.charge = -5

r = model.reactions.get_by_id('AOXSr2').copy()
r.id = 'AOXSp'
model.add_reactions([r])
r.add_metabolites({'ACP_c': -1.0, 'coa_c': 1.0, 'h_c': -1.0})
for m in r.metabolites:
    if not m.id.replace('_c','_x') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_x')
        m2.compartment = 'x'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_x'): r.get_coefficient(m.id)})

r = sce.reactions.get_by_id('DBTS').copy()
r.gene_reaction_rule = '12731'
model.add_reactions([r])

for x in ['AMAOTr','DBTS','BTS5']:
    r = model.reactions.get_by_id(x)
    r.id = r.id + 'm'
    for m in r.metabolites:
        if not m.id.replace('_c','_m') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_m')
            m2.compartment = 'm'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})

model.remove_reactions(['AOXSr2','BTSr','yli_R1487','yli_R1488','yli_R1489','yli_R1490'],remove_orphans=True)

#### GLUSy

In [22]:
for r in sorted(model.genes.get_by_id('15713').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12248').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9856').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

GLUDy glu__L_c + h2o_c + nadp_c <=> akg_c + h_c + nadph_c + nh4_c 12248 or (b3213 and 15713)
GLUS akg_h + gln__L_h + h_h + nadh_h --> 2.0 glu__L_h + nad_h 15713
GLUS_ferr akg_h + 2.0 fdxrd_h + gln__L_h --> 2.0 fdxox_h + 2.0 glu__L_h + 2.0 h_h (CRv4_Au5_s16_g6229_t1 and 15713) or (CRv4_Au5_s17_g7064_t1 and 15713) or (CRv4_Au5_s3_g10824_t1 and 15713) or (CRv4_Au5_s6_g13523_t1 and 15713) or (CRv4_Au5_s7_g14133_t1 and 15713)
GLUS_nadph akg_h + gln__L_h + h_h + nadph_h --> 2.0 glu__L_h + nadp_h 15713
GLUSx akg_c + gln__L_c + h_c + nadh_c --> 2.0 glu__L_c + nad_c 15713
GLUSy akg_c + gln__L_c + h_c + nadph_c --> 2.0 glu__L_c + nadp_c (PP_5075 and 15713) or (b3213 and 15713)
yli_R0002 akg_c + gln__L_c + h_c + nadph_c --> glu__L_c + nadp_c 15713

GDHm glu__L_m + h2o_m + nad_m <=> akg_m + h_m + nadh_m + nh4_m 12248
GLUDy glu__L_c + h2o_c + nadp_c <=> akg_c + h_c + nadph_c + nh4_c 12248 or (b3213 and 15713)
GLUDym glu__L_m + h2o_m + nadp_m <=> akg_m + h_m + nadph_m + nh4_m 12248

GLUDxi glu__L_c 

15713 cyto GLT1 glutamate synthase NAD -> GLUSx  
12248 cyto GDH1, GDH3 glutamate dehydrogenase NADP -> GLUDy  
9856 cyto GDH2 glutamate dehydrogenase NAD -> GLUDxi

Change GLUDy genes to '12248'  
Remove GLUS, GLUS_ferr, GLUS_nadph, GLUSy, yli_R0002, GDHm, GLUDym

In [23]:
model.reactions.get_by_id('GLUDy').gene_reaction_rule = '12248'
model.remove_reactions(['GLUS','GLUS_ferr','GLUS_nadph','GLUSy','yli_R0002','GDHm','GLUDym'],remove_orphans=True)

#### mlthf / GCC2cm

In [24]:
for r in sorted(model.metabolites.get_by_id('mlthf_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

yli_R1377 co2_x + mlthf_x + nadh_x + nh4_x --> gly_x + nad_x + thf_x 10205
yli_R1378 co2_x + mlthf_x + nadh_x + nh4_x --> gly_x + nad_x + thf_x 10040
yli_R1387 gly_x + h2o_x + mlthf_x <=> ser__L_x + thf_x 9667
yli_R1425 co2_x + mlthf_x + nadh_x + nh4_x --> gly_x + nad_x + thf_x 12898


In [25]:
for r in sorted(model.genes.get_by_id('10205').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10040').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9667').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12898').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

GCC2am gly_m + h_m + lpam_m <=> alpam_m + co2_m 10040 and 10205 and 12898 and 15184
GCC2bim alpam_m + thf_m --> dhlam_m + mlthf_m + nh4_m 10040 and 10205 and 12898 and 15184
GCC2cm dhlam_m + nad_m <=> h_m + lpam_m + nadh_m 10040 and 10205 and 12898 and 15184
GCC2cm_copy1 dhlam_m + nad_m <=> h_m + lpam_m + nadh_m 10040 and 10205 and 12898 and 15184
GCCam gly_m + h_m + lpro_m <=> alpro_m + co2_m 10205 or (10040 and 10205 and 12898 and 15184)
GCCbim alpro_m + thf_m --> dhlpro_m + mlthf_m + nh4_m 12898 or (10040 and 10205 and 12898 and 15184)
GCCcm dhlpro_m + nad_m <=> h_m + lpro_m + nadh_m 10040 or (10040 and 10205 and 12898 and 15184)
GLYCL gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c 10040 and 10205 and 12898 and 15184
GLYCLm gly_m + nad_m + thf_m --> co2_m + mlthf_m + nadh_m + nh4_m 12898 or (CRv4_Au5_s12_g4121_t1 and 14894) or (12898 and 14894) or (10040 and 10205 and 12898 and 15184)
GLYDHD gly_m + lpro_m --> alpro_m + co2_m 10205 and 15184
THFATm h2o_m + methf_m --> 5f

10040 mito 9 cyto 8 LPD1  
glycine cleavage complex  
12898 mito GCV1  
10205 mito GCV2  
15184 mito GCV3  
alpha keto-glutarate dehydrogenase  
10007 mito KGD1  
12116 mito KGD2  
oxoadipate dehydrogenase  
9724 nucl KGD1 probable 2-oxoglutarate dehydrogenase E1 component DHKTD1  
pyruvate dehydrogenase  
13630 mito PDA1  
13948 cysk PDB1  
13722 mito PDX1  
14126 cyto LAT1  
serine hydroxymethyltransferase  
9667 cyto SHM1, SHM2 (GHMT2r)  
L-threonine aldolase  
16182 cyto GLY1  
9222 mito GLY1, hydroxytrimethyllysine aldolase  

14894 is IBA57 protein involved in incorporating iron-sulfur clusters into proteins

GCC2am, GCC2bim, GCC2cm (GCC2cm_copy1, GCC2cm_copy2) is GLYCLm  
GCCam, GCCbim, GCCcm is GLYCLm

MTAM is wrong (FTHFCLm is correct with ATP)  
Replace FTHFCL with FTHFCLm and change genes to '12562' mito fau1  
MTAM_nh4 is GCCbim

THFAT/THFATm reaction is catalyzed by serine hydroxymethyltransferase, not gcv or pda/pdb  
It is not known that yeast SHM has THFAT activity, and S. cerevisiae biocyc does not have this reaction.  
THFATm and FTHFCLm make a futile cycle, and the role of 5fthf is not clear.

GHMT2r only cyto SHMT is present in Rhodo  
GHMT3/GHMT3m is part of L-carnitine biosynthesis -> by mito GLY1 (paperblast to hydroxytrimethyllysine aldolase)  
https://www.ncbi.nlm.nih.gov/pubmed/19289605  
S. cer cannot synthesize carnitine, is it important for lipid for its role in shuttle?  
GHMT3/GHMT3m has incorrect stoichiometry (proton), replace with HTMLA_m from iLB1027_lipid

ALATA_D2, ALATA_L2 is inactivation of the enzyme in E. coli, activity is low  
https://onlinelibrary.wiley.com/doi/full/10.1046/j.0014-2956.2001.02606.x

Change GLYCLm genes to '10040 and 10205 and 12898 and 15184'  
Remove GCC2am, GCC2bim, GCC2cm, GCC2cm_copy1, GCC2cm_copy2, GCCam, GCCbim, GCCcm  
Remove GLYCL, GLYCL_2, GLYDHD, MTAM, MTAM_nh4, THFAT, THFATm, yli_R1377, yli_R1378, yli_R1425

Add FTHFCLm and change genes to '12562'  
Remove FTHFCL

Change AKGDm genes to '10007 and 10040 and 12116'  
Change 2OXOADOXm genes to '10040 and 12116 and 9274'  
Remove AKGDH, AKGDam, AKGDbm

Change PDHm genes to '10040 and 13630 and 13722 and 13948 and 14126'  
Remove PDH, PDHcr, yli_R0381, yli_R1419

Remove GHMT2rm, GHMT3, GHMT3m, ALATA_D2, ALATA_L2, yli_R1387  

Change THRA genes to '16182'  
Change THRA2 genes to '16182'  
Add HTMLA_m from iLB1027_lipid, and change genes to '9222'

In [26]:
model.reactions.get_by_id('GLYCLm').gene_reaction_rule = '10040 and 10205 and 12898 and 15184'
model.remove_reactions(['GCC2am','GCC2bim','GCC2cm','GCC2cm_copy1','GCC2cm_copy2','GCCam','GCCbim','GCCcm'],remove_orphans=True)
model.remove_reactions(['GLYCL','GLYCL_2','GLYDHD','MTAM','MTAM_nh4','THFAT','THFATm','yli_R1377','yli_R1378','yli_R1425'],remove_orphans=True)
r = sce.reactions.get_by_id('FTHFCLm').copy()
r.gene_reaction_rule = '12562'
model.add_reactions([r])
model.remove_reactions(['FTHFCL'],remove_orphans=True)
model.reactions.get_by_id('AKGDm').gene_reaction_rule = '10007 and 10040 and 12116'
model.reactions.get_by_id('2OXOADOXm').gene_reaction_rule = '10040 and 12116 and 9274'
model.remove_reactions(['AKGDH','AKGDam','AKGDbm'],remove_orphans=True)
model.reactions.get_by_id('PDHm').gene_reaction_rule = '10040 and 13630 and 13722 and 13948 and 14126'
model.remove_reactions(['PDH','PDHcr','yli_R0381','yli_R1419'],remove_orphans=True)
model.remove_reactions(['GHMT2rm','GHMT3','GHMT3m','ALATA_D2','ALATA_L2','yli_R1387'],remove_orphans=True)
model.reactions.get_by_id('THRA').gene_reaction_rule = '16182'
model.reactions.get_by_id('THRA2').gene_reaction_rule = '16182'
r = ptri.reactions.get_by_id('HTMLA_m').copy()
r.gene_reaction_rule = '9222'
model.add_reactions([r])

In [27]:
for r in sorted(model.genes.get_by_id('10040').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10205').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12898').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15184').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12562').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10007').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12116').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9274').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13630').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13722').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13948').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('14126').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9667').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16182').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9222').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

2OXOADOXm 2oxoadp_m + coa_m + nad_m --> co2_m + glutcoa_m + nadh_m 10040 and 12116 and 9274
AKGDm akg_m + coa_m + nad_m --> co2_m + nadh_m + succoa_m 10007 and 10040 and 12116
GLYCLm gly_m + nad_m + thf_m --> co2_m + mlthf_m + nadh_m + nh4_m 10040 and 10205 and 12898 and 15184
OIVD1m 4mop_m + coa_m + nad_m --> co2_m + ivcoa_m + nadh_m (10040 and 11183 and 12086 and 15436) or (10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12086 and 15436) or (10040 and 11188 and 12566 and 15436)
OIVD2m 3mob_m + coa_m + nad_m --> co2_m + ibcoa_m + nadh_m (10040 and 11183 and 12086 and 15436) or (10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12086 and 15436) or (10040 and 11188 and 12566 and 15436)
OIVD3m 3mop_m + coa_m + nad_m --> 2mbcoa_m + co2_m + nadh_m (10040 and 11183 and 12086 and 15436) or (10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12086 and 15436) or (10040 and 11188 and 12566 and 15436)
PDHm coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m 10040

In [28]:
for r in sorted(model.metabolites.get_by_id('4hthr_c').reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('phthr_c').reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)

4HTHRA 4hthr_c <=> gcald_c + gly_c 16182 or 9222
4HTHRK 4hthr_c + atp_c --> adp_c + h_c + phthr_c 8651
4HTHRS h2o_c + phthr_c --> 4hthr_c + pi_c 9742

4HTHRK 4hthr_c + atp_c --> adp_c + h_c + phthr_c 8651
4HTHRS h2o_c + phthr_c --> 4hthr_c + pi_c 9742


In [29]:
for r in sorted(model.genes.get_by_id('8651').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9742').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

4HTHRK 4hthr_c + atp_c --> adp_c + h_c + phthr_c 8651
HSK atp_c + hom__L_c --> adp_c + h_c + phom_c 8651
HSK_1 atp_h + hom__L_h --> adp_h + h_h + phom_h 8651

4HTHRS h2o_c + phthr_c --> 4hthr_c + pi_c 9742
THRS h2o_c + phom_c --> pi_c + thr__L_c 9742
THRS_1 h2o_h + phom_h --> pi_h + thr__L_h 9742


Remove FOMETRi, AKGDa, GLCOASYNT, yli_R0428, yli_R0788, yli_R1575, yli_R1576  
Remove AKGDHe2r, AKGDb, OXOADLR, yli_R0784, yli_R1532  
Remove PDHam1hi, PDHam1mi, PDHam2hi, PDHam2mi, yli_R0357, yli_R0377, yli_R0848, yli_R1375, PDHe2r, yli_R0374  
Remove THRA_1, 4HTHRA, 4HTHRK, HSK_1, 4HTHRS, THRS_1  
4HTHRA is by E. coli itaE, not by glyA (GLY1) and not in S. cer biocyc. Similar for 4HTHRK, 4HTHRS

In [30]:
model.remove_reactions(['FOMETRi','AKGDa','GLCOASYNT','yli_R0428','yli_R0788','yli_R1575','yli_R1576'],remove_orphans=True)
model.remove_reactions(['AKGDHe2r','AKGDb','OXOADLR','yli_R0784','yli_R1532'],remove_orphans=True)
model.remove_reactions(['PDHam1hi','PDHam1mi','PDHam2hi','PDHam2mi','yli_R0357','yli_R0377','yli_R0848','yli_R1375','PDHe2r','yli_R0374'],remove_orphans=True)
model.remove_reactions(['THRA_1','4HTHRA','4HTHRK','HSK_1','4HTHRS','THRS_1'],remove_orphans=True)

In [31]:
# Search for reactions involiving tmlys, 3htmelys 
for r in sorted(model.genes.get_by_id('14530').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16853').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9180').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9222').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

LYSMTF1n amet_n + peplys_n --> Nmelys_n + ahcys_n 14530
LYSMTF2n Nmelys_n + amet_n --> Ndmelys_n + ahcys_n 14530
LYSMTF3n Ndmelys_n + amet_n --> Ntmelys_n + ahcys_n 14530

PLYSPSer Ntmelys_r + h2o_r --> pepslys_r + tmlys_r (Spcs2 and Spcs3 and 16853 and 9881) or (SEC11A and SPCS2 and 10517 and 16853 and 9881)

TMLYSOX akg_c + o2_c + tmlys_c --> 3htmelys_c + co2_c + succ_c 9180

HTMLA_m 3htmelys_m --> 4tmeabut_m + gly_m 9222


L-carnitine biosynthesis  
14530	mito 22, nucl 2, cyto 2, cyto_nucl 2	K11427: DOT1L, DOT1; histone-lysine N-methyltransferase, H3 lysine-79 specific  
9180 mito TMLHE, trimethyllysine dioxygenase -> replace TMLYSOX with TMLOX_m from iLB1027_lipid  
9222 mito GLY1, hydroxytrimethyllysine aldolase  
13426 mito blast of 4-trimethylammoniobutyraldehyde dehydrogenase ALDH9A1 hit -> TMABDH1_m  
15060 cyto 10, nucl 7, pero 7, extr 2 BBOX1, gamma-butyrobetaine dioxygenase -> BBHOX exist in BiGG, but assume mito and use GBBOX_m  
Longer RNA-extended gene model is predicted to be mitochondrial  
Remove upstream reactions (by 14530) since it is not clear where tmlys_c is coming from

TMLOX_m akg_m + o2_m + tmlys_m ⇌ co2_m + succ_m + 3htmelys_m 9180  
HTMLA_m 3htmelys_m --> 4tmeabut_m + gly_m 9222  
TMABDH1_m h2o_m + nad_m + 4tmeabut_m ⇌ 2.0 h_m + nadh_m + gbbtn_m 13426  
GBBOX_m akg_m + o2_m + gbbtn_m → co2_m + crn_m + succ_m 15060

Add TMLOX_m from iLB1027_lipid and change genes to '9180'  
Add TMABDH1_m from iLB1027_lipid and change genes to '13426'  
Add GBBOX_m from iLB1027_lipid and change genes to '15060'  
Remove TMLYSOX, LYSMTF1n, LYSMTF2n, LYSMTF3n, PLYSPSer

In [32]:
r = ptri.reactions.get_by_id('TMLOX_m').copy()
r.gene_reaction_rule = '9180'
model.add_reactions([r])
r = ptri.reactions.get_by_id('TMABDH1_m').copy()
r.gene_reaction_rule = '13426'
model.add_reactions([r])
r = ptri.reactions.get_by_id('GBBOX_m').copy()
r.gene_reaction_rule = '15060'
model.add_reactions([r])
model.remove_reactions(['TMLYSOX','LYSMTF1n','LYSMTF2n','LYSMTF3n','PLYSPSer'],remove_orphans=True)

In [33]:
for r in sorted(model.genes.get_by_id('12566').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15436').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11183').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12086').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11188').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

MOD 3mob_m + h_m + thmpp_m --> 2mhop_m + co2_m 12566 and 15436
MOD_2mbdhl 2mhob_m + lpam_m --> 2mbdhl_m + thmpp_m 12566 and 15436
MOD_2mhop 2mhop_m + lpam_m --> 2mpdhl_m + thmpp_m 12566 and 15436
MOD_3mhtpp 3mhtpp_m + lpam_m --> 3mbdhl_m + thmpp_m 12566 and 15436
MOD_3mop 3mop_m + h_m + thmpp_m --> 2mhob_m + co2_m 12566 and 15436
MOD_4mop 4mop_m + h_m + thmpp_m --> 3mhtpp_m + co2_m 12566 and 15436
OIVD1m 4mop_m + coa_m + nad_m --> co2_m + ivcoa_m + nadh_m (10040 and 11183 and 12086 and 15436) or (10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12086 and 15436) or (10040 and 11188 and 12566 and 15436)
OIVD1r 4mop_c + coa_c + nad_c <=> co2_c + ivcoa_c + nadh_c (PP_4404 and 11183 and 12566 and 15436) or (PP_4404 and 11188 and 12566 and 15436)
OIVD2 3mob_c + coa_c + nad_c --> co2_c + ibcoa_c + nadh_c (PP_4404 and 11183 and 12566 and 15436) or (PP_4404 and 11188 and 12566 and 15436)
OIVD2m 3mob_m + coa_m + nad_m --> co2_m + ibcoa_m + nadh_m (10040 and 11183 and 12086 and 15436)

branched-chain alpha-keto acid dehydrogenase (missing in S. cer)  
12566 mito bkdA1 Branched chain alpha-keto acid dehydrogenase E1, alpha subunit  
15436 mito bkdA2 Branched chain alpha-keto acid dehydrogenase E1, beta subunit  
11183 mito bkdB Dihydrolipoamide transacylase (alpha-keto acid dehydrogenase E2 subunit)  
extra cyto subunits?  
12086 cyto ortholog of E1, Short-chain acyl-CoA dehydrogenase (butyryl) has cytochrome b5-like domain -> fatty acid oxidation  
11188 cyto Dihydrolipoamide transacylase (alpha-keto acid dehydrogenase E2 subunit) -> keep as isozyme for now

Keep mito OIVDs (OIVD1m, OIVD2m, OIVD3m), and change genes to '(10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12566 and 15436)'  

BCAA additional reactions, e.g., OBDHm in S. cer is missing genes  
Add OBDHm from iMM904 and change genes to OIVD1m genes

Remove cyto OIVDs (OIVD1r, OIVD2, OIVD3), MOD, MOD_2mbdhl, MOD_2mhop, MOD_3mhtpp, MOD_3mop, MOD_4mop  
Remove yli_R0937, yli_R0938, yli_R0939, yli_R1587, yli_R1590, yli_R1591, yli_R1592  
Remove DHRT_2mbcoa, DHRT_ibcoa, DHRT_ivcoa, yli_R0878, yli_R0879, yli_R0880

In [34]:
model.reactions.get_by_id('OIVD1m').gene_reaction_rule = '(10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12566 and 15436)'
model.reactions.get_by_id('OIVD2m').gene_reaction_rule = '(10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12566 and 15436)'
model.reactions.get_by_id('OIVD3m').gene_reaction_rule = '(10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12566 and 15436)'
r = sce.reactions.get_by_id('OBDHm').copy()
r.gene_reaction_rule = '(10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12566 and 15436)'
model.add_reactions([r])
model.remove_reactions(['OIVD1r','OIVD2','OIVD3','MOD','MOD_2mbdhl','MOD_2mhop','MOD_3mhtpp','MOD_3mop','MOD_4mop'],remove_orphans=True)
model.remove_reactions(['yli_R0937','yli_R0938','yli_R0939','yli_R1587','yli_R1590','yli_R1591','yli_R1592'],remove_orphans=True)
model.remove_reactions(['DHRT_2mbcoa','DHRT_ibcoa','DHRT_ivcoa','yli_R0878','yli_R0879','yli_R0880'],remove_orphans=True)

In [35]:
for r in sorted(model.genes.get_by_id('15685').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9800').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACAS_2ahbut 2ahethmpp_h + 2obut_h --> 2ahbut_h + thmpp_h 15685 and 9800
ACHBS 2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c (b3670 and 15685) or (15685 and 9800)
ACHBSm 2obut_m + h_m + pyr_m --> 2ahbut_m + co2_m 15685 and 9800
ACLS h_c + 2.0 pyr_c --> alac__S_c + co2_c (b3670 and 15685) or (15685 and 9800)
ACLSm h_m + 2.0 pyr_m --> alac__S_m + co2_m 15685 and 9800
APLh 2ahethmpp_h + pyr_h --> alac__S_h + thmpp_h 15685 and 9800
APLm 2ahethmpp_m + pyr_m --> alac__S_m + thmpp_m 15685 and 9800
PPATDh h_h + 2.0 pyr_h --> alac__S_h + co2_h 15685 and 9800
yli_R0861 2ahethmpp_m + pyr_m --> alac__S_m + 3.0 h_m + thmpp_m 15685 and 9800
yli_R0862 2ahethmpp_m + 2obut_m --> 2ahbut_m + 3.0 h_m + thmpp_m 15685 and 9800
yli_R1588 2obut_m + pyr_m --> 2ahbut_m + co2_m 15685 and 9800

ACAS_2ahbut 2ahethmpp_h + 2obut_h --> 2ahbut_h + thmpp_h 15685 and 9800
ACHBS 2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c (b3670 and 15685) or (15685 and 9800)
ACHBSm 2obut_m + h_m + pyr_m --> 2ahbut_m + co2_m 15685 and 9800
AC

15685 mito ILV2  
9800 mito ILV6  
ILV2 and ILV6 catalyze ACHBSm and ACLSm  
Remove ACAS_2ahbut, ACHBS, ACLS, APLh, APLm, PPATDh, yli_R0861, yli_R0862, yli_R1588

In [36]:
model.remove_reactions(['ACAS_2ahbut','ACHBS','ACLS','APLh','APLm','PPATDh','yli_R0861','yli_R0862','yli_R1588'],remove_orphans=True)

In [37]:
for r in sorted(model.metabolites.get_by_id('2ahethmpp_m').reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('2ahethmpp_c').reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)

PDCm 2ahethmpp_m --> acald_m + thmpp_m 15791

yli_R0364 2ahethmpp_c --> acald_c + 3.0 h_c + thmpp_c 15791


In [38]:
for r in sorted(model.genes.get_by_id('15791').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

3MOBDC 3mob_c + h_c --> 2mppal_c + co2_c 15791
3MOPDC 3mop_c + h_c --> 2mbald_c + co2_c 15791
4MOPDC 4mop_c + h_c --> 3mbald_c + co2_c 15791
ACALDCD 2.0 acald_c --> actn__R_c 15791
INDPYRD h_c + indpyr_c <=> co2_c + id3acald_c 15791
PDCm 2ahethmpp_m --> acald_m + thmpp_m 15791
PPYRDC h_c + phpyr_c --> co2_c + pacald_c 15791
PYRDC h_c + pyr_c --> acald_c + co2_c 15791
PYRDC2 acald_c + h_c + pyr_c --> actn__R_c + co2_c 15791
PYRDC_1 h_m + pyr_m --> acald_m + co2_m 15791
yli_R0364 2ahethmpp_c --> acald_c + 3.0 h_c + thmpp_c 15791


15791 cyto PDC  
Remove PDCm, PYRDC_1, yli_R0364

In [39]:
model.remove_reactions(['PDCm','PYRDC_1','yli_R0364'],remove_orphans=True)

#### 2ippm

In [40]:
for r in sorted(model.genes.get_by_id('14914').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

IPPMIa 3c2hmp_c <=> 2ippm_c + h2o_c 14914 or (CRv4_Au5_s6_g12448_t1 and 14914) or (PP_1986 and 14914) or (b0071 and 14914)
IPPMIb 2ippm_c + h2o_c <=> 3c3hmp_c 14914 or (CRv4_Au5_s6_g12448_t1 and 14914) or (PP_1986 and 14914) or (b0071 and 14914)
yli_R1465 3c3hmp_m <=> 2ippm_m + h2o_m 14914
yli_R1466 2ippm_m + h2o_m <=> 3c2hmp_m 14914
yli_R7859 2ippm_m + h2o_m <=> 3c2hmp_m 14914
yli_R8859 3c3hmp_m <=> 2ippm_m + h2o_m 14914


In [41]:
# 14914 is cytosolic, and contains both large (leuC) and small (leuD) subunits
model.reactions.get_by_id('IPPMIa').gene_reaction_rule = '14914'
model.reactions.get_by_id('IPPMIb').gene_reaction_rule = '14914'
model.remove_reactions(['yli_R1465','yli_R1466','yli_R7859','yli_R8859'], remove_orphans=True)

#### cer1_24

In [42]:
for r in sorted(model.genes.get_by_id('9664').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15314').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

CERH124_copy2 cer1_24_c + h_c + nadph_c + o2_c --> cer2_24_c + h2o_c + nadp_c 9664
CERH126_copy2 cer1_26_c + h_c + nadph_c + o2_c --> cer2_26_c + h2o_c + nadp_c 9664
CERS324 cer2_24_c + h_c + nadph_c + o2_c --> cer3_24_c + h2o_c + nadp_c 9664
CERS326 cer2_26_c + h_c + nadph_c + o2_c --> cer3_26_c + h2o_c + nadp_c 9664

44MZYMMO 44mzym_c + 2.0 h_c + 3.0 nadph_c + 3.0 o2_c <=> 4mzym_int1_c + 4.0 h2o_c + 3.0 nadp_c 15314
CERH124_copy1 cer1_24_c + h_c + nadph_c + o2_c --> cer2_24_c + h2o_c + nadp_c 15314
CERH126_copy1 cer1_26_c + h_c + nadph_c + o2_c --> cer2_26_c + h2o_c + nadp_c 15314
PSPHS h_c + nadph_c + o2_c + sphgn_c --> h2o_c + nadp_c + psphings_c 15314
yli_R0702 h_c + nadph_c + o2_c + yli_M05742_c --> h2o_c + nadp_c + yli_M05749_c 15314
yli_R0703 h_c + nadph_c + o2_c + yli_M05741_c --> h2o_c + nadp_c + yli_M05748_c 15314
yli_R0704 h_c + nadph_c + o2_c + yli_M05749_c --> h2o_c + nadp_c + yli_M05725_c 15314
yli_R0705 h_c + nadph_c + o2_c + yli_M05748_c --> h2o_c + nadp_c + yli_M05724

Sphingolipid biosynthesis  
https://onlinelibrary.wiley.com/doi/pdf/10.1111/tra.12239  
https://www.sciencedirect.com/science/article/pii/S0022283616303746  
https://link.springer.com/content/pdf/10.1007%2F978-3-319-43676-0_21-1.pdf  
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3683901/  
https://pdfs.semanticscholar.org/e239/05c20c98c108bd8a7613dde68d283e7cba22.pdf  
SERPT  
10303 cyto LCB1 (SPOTS complex in ER membrane) serine palmitoyltransferase  
9425 cyto LCB2 (SPOTS complex in ER membrane) serine palmitoyltransferase  
9394 mito Small subunit of serine palmitoyltransferase-like -> TSC3  
3DSPHR  
9979 extr TSC10 (lipid droplet)  
PSPHS  
15314 mito 9, plas 5, E.R. 4 SUR2 (uses cytochrome b5)  
CERS124/CERS124er  
11391 plas LAC1, LAG1 (ER) ceramide synthase  
15168 plas ceramide synthase  
can't find LIP1  
SBPP1/SBPP1er  
13412 plas LCB3, YSR3 (ER) check for isozyme  
16352 DAG/PA phosphatase LPP1?  
12927 AUR1 Phosphatidylinositol:ceramide phosphoinositol transferase  
13172 IPT1 Phosphatidylinositol:ceramide phosphoinositol transferase  
13087 DPP1 DAG/PA phosphatase  
15202 PA phosphatase 3 LPP3 in Arabidopsis  
14049 dolichyldiphosphatase  
SLCBK1/SLCBK2  
10391 cyto LCB4 (plasma, ER, golgi), LCB5 (golgi) Sphingosine kinase  
16228 cyto Sphingosine kinase  
CERH124_copy2/CERS324  
9664 mito 9 pero 7 cyto_mito 7 mito_nucl 7 SCS7 (membrane, ER)  
PSPHPL  
11925 DPL1 (ER) sphinganine-1-phosphate aldolase

10303	cyto 11, cyto_nucl 9.5, mito 7, nucl 6	K00654: SPT; serine palmitoyltransferase	GKK*  
9425	cyto 11.5, mito 10, cyto_nucl 9, nucl 5.5	K00654: SPT; serine palmitoyltransferase	EHA*  
9394	mito 12, extr 7, mito_nucl 7	HMMPfam:Protein of unknown function (DUF3317):PF11779	ELL*  
9979	extr 16, nucl 3, mito 3, mito_nucl 3	K04708: E1.1.1.102; 3-dehydrosphinganine reductase	VVG*  
15314	mito 9, plas 5, E.R. 4, extr 3, golg 3, cyto 2	K04713: SUR2; sphinganine C4-monooxygenase	KTE*  
11391	plas 22, mito 2, cyto 1, pero 1, E.R. 1, mito_nucl 1, cyto_pero 1	K04709: LAG1; Acyl-CoA-dependent ceramide synthase	KKR*  
15168	plas 20, mito 3, vacu 2	K04709: LAG1; Acyl-CoA-dependent ceramide synthase	KER*  
13412	plas 26	K04717: SGPP2; sphingosine-1-phosphate phosphotase 2	SVR*  
16352	plas 14, extr 6, E.R. 3, mito 2, vacu 2	KOG3030: Lipid phosphate phosphatase and related enzymes of the PAP2 family	AVV*  
12927	plas 22, mito 2, vacu 2	HMMPfam:PAP2 superfamily:PF01569,SMART:Acid phosphatase homologues:SM00014,SUPERFAMILY::SSF48317	RRD*  
13172	extr 10, mito 7, plas 4, cyto_mito 4	HMMPfam:PAP2 superfamily:PF01569	SLA*  
13087	plas 23, mito 2	K18693: DPP1; diacylglycerol diphosphate phosphatase / phosphatidate phosphatase	GYY*  
15202	plas 27	KOG3030: Lipid phosphate phosphatase and related enzymes of the PAP2 family	RMV*  
14049	mito 13, extr 12	KOG3146: Dolichyl pyrophosphate phosphatase and related acid phosphatases	GEL*  
10391	cyto 13.5, cyto_nucl 12, nucl 5.5, pero 4, mito 3	K04718: SPHK; sphingosine kinase	GFD*  
16228	cyto 12, nucl 8, mito 5	HMMPfam:Diacylglycerol kinase catalytic domain:PF00781,ProSiteProfiles:DAG-kinase catalytic (DAGKc) domain profile.:PS50146,SUPERFAMILY::SSF111331	EKE*  
9664	mito 9, pero 7, cyto_mito 7, mito_nucl 7	K19703: FA2H, SCS7; 4-hydroxysphinganine ceramide fatty acyl 2-hydroxylase	AKA*  
11925	cyto 17.5, cyto_nucl 9.5, mito 9	K01634: SGPL1, DPL1; sphinganine-1-phosphate aldolase	LYA*

In [43]:
for r in sorted(model.genes.get_by_id('10303').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9425').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9979').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15314').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11391').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15168').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13412').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10391').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9664').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11925').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

GLYATx accoa_x + gly_x <=> 2aobut_x + coa_x + h_x 10303
SERPT h_c + pmtcoa_c + ser__L_c --> 3dsphgn_c + co2_c + coa_c 10303 or 9425 or (10303 and 9425) or (YBR058C_A and 10303 and 9425)
yli_R1438 h_r + pmtcoa_r + ser__L_r --> 3dsphgn_r + co2_r + coa_r 10303 or 9425

SERPT h_c + pmtcoa_c + ser__L_c --> 3dsphgn_c + co2_c + coa_c 10303 or 9425 or (10303 and 9425) or (YBR058C_A and 10303 and 9425)
yli_R1438 h_r + pmtcoa_r + ser__L_r --> 3dsphgn_r + co2_r + coa_r 10303 or 9425

3DSPHR 3dsphgn_c + h_c + nadph_c --> nadp_c + sphgn_c 9979
yli_R1440 3dsphgn_r + h_r + nadph_r --> nadp_r + sphgn_r 9979

44MZYMMO 44mzym_c + 2.0 h_c + 3.0 nadph_c + 3.0 o2_c <=> 4mzym_int1_c + 4.0 h2o_c + 3.0 nadp_c 15314
CERH124_copy1 cer1_24_c + h_c + nadph_c + o2_c --> cer2_24_c + h2o_c + nadp_c 15314
CERH126_copy1 cer1_26_c + h_c + nadph_c + o2_c --> cer2_26_c + h2o_c + nadp_c 15314
PSPHS h_c + nadph_c + o2_c + sphgn_c --> h2o_c + nadp_c + psphings_c 15314
yli_R0702 h_c + nadph_c + o2_c + yli_M05742_c --> h2o_c 

Change yli_R1438 to SERPTer  
Change SERPTer genes to '10303 and 9394 and 9425'  
Remove GLYATx, yli_R1438  
Change yli_R1440 to 3DSPHRer  
Remove 3DSPHR  
Change CERH124_copy1 to CERH124er, and change metabolites from c to r  
Change CERH126_copy1 to CERH126er, and change metabolites from c to r  
Change yli_R1441 to PSPHSer  
Remove PSPHS, yli_R0702, yli_R0703, yli_R0704, yli_R0705, yli_R1571  
Change CERS124er genes to '11391 or 15168'  
Change CERS126er genes to '11391 or 15168'  
Change CERS224er genes to '11391 or 15168'  
Change CERS226er genes to '11391 or 15168'  
Remove CERS124, CERS126, CERS224, CERS226, yli_R0698, yli_R0699, yli_R0700, yli_R0701, yli_R1570  
Remove SBPP1  
Change SLCBK1 to SLCBK1er, and change genes to '10391 or 16228'  
Change SLCBK2 to SLCBK2er, and change genes to '10391 or 16228'  
Remove SPHK21c, SGOR, yli_R1439, yli_R1569  
Change CERH124_copy2 to CERS2p24, change metabolites from c to r, and change cer2_24 to cer2p_24  
Change CERH126_copy2 to CERS2p26, change metabolites from c to r, and change cer2_26 to cer2p_26  
Change CERS324 to CERS324er, and change metabolites from c to r  
Change CERS326 to CERS326er, and change metabolites from c to r  
Change PSPHPL to PSPHPLer, and change metabolites from c to r  
Change SPHPL to SPHPLer, and change metabolites from c to r  
Remove SGPL11r, SGPL12r, SGPL13, yli_R1567, yli_R1568

In [44]:
model.reactions.get_by_id('yli_R1438').id = 'SERPTer'
model.reactions.get_by_id('SERPTer').gene_reaction_rule = '10303 and 9394 and 9425'
model.remove_reactions(['GLYATx','SERPT'], remove_orphans=True)
model.reactions.get_by_id('yli_R1440').id = '3DSPHRer'
model.remove_reactions(['3DSPHR'], remove_orphans=True)
r = model.reactions.get_by_id('CERH124_copy1').copy()
model.reactions.get_by_id('CERH124_copy1').id = 'CERH124er'
for m in r.metabolites:
    model.reactions.get_by_id('CERH124er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
r = model.reactions.get_by_id('CERH126_copy1').copy()
model.reactions.get_by_id('CERH126_copy1').id = 'CERH126er'
for m in r.metabolites:
    model.reactions.get_by_id('CERH126er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.reactions.get_by_id('yli_R1441').id = 'PSPHSer'
model.remove_reactions(['PSPHS','yli_R0702','yli_R0703','yli_R0704','yli_R0705','yli_R1571'], remove_orphans=True)
model.reactions.get_by_id('CERS124er').gene_reaction_rule = '11391 or 15168'
model.reactions.get_by_id('CERS126er').gene_reaction_rule = '11391 or 15168'
model.reactions.get_by_id('CERS224er').gene_reaction_rule = '11391 or 15168'
model.reactions.get_by_id('CERS226er').gene_reaction_rule = '11391 or 15168'
model.remove_reactions(['CERS124','CERS126','CERS224','CERS226','yli_R0698','yli_R0699','yli_R0700','yli_R0701','yli_R1570','SBPP1'], remove_orphans=True)
r = model.reactions.get_by_id('SLCBK1').copy()
model.reactions.get_by_id('SLCBK1').id = 'SLCBK1er'
model.reactions.get_by_id('SLCBK1er').gene_reaction_rule = '10391 or 16228'
for m in r.metabolites:
    model.reactions.get_by_id('SLCBK1er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
r = model.reactions.get_by_id('SLCBK2').copy()
model.reactions.get_by_id('SLCBK2').id = 'SLCBK2er'
model.reactions.get_by_id('SLCBK2er').gene_reaction_rule = '10391 or 16228'
for m in r.metabolites:
    model.reactions.get_by_id('SLCBK2er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.remove_reactions(['SPHK21c','SGOR','yli_R1439','yli_R1569'], remove_orphans=True)
m1 = model.metabolites.get_by_id('cer2_24_c').copy()
m2 = model.metabolites.get_by_id('cer2_26_c').copy()
m1.id = 'cer2p_24_r'
m1.compartment = 'r'
m2.id = 'cer2p_26_r'
m2.compartment = 'r'
m1.name = 'Ceramide 2p (Sphinganine:n-C24:0OH)'
m2.name = 'Ceramide 2p (Sphinganine:n-C26:0OH)'
model.add_metabolites([m1,m2])
r = model.reactions.get_by_id('CERH124_copy2').copy()
model.reactions.get_by_id('CERH124_copy2').id = 'CERS2p24er'
model.reactions.get_by_id('CERS2p24er').name = 'Ceramide 2p synthase  24C'
for m in r.metabolites:
    model.reactions.get_by_id('CERS2p24er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.reactions.get_by_id('CERS2p24er').add_metabolites({'cer2_24_r': -1.0, 'cer2p_24_r': 1.0})
r = model.reactions.get_by_id('CERH126_copy2').copy()
model.reactions.get_by_id('CERH126_copy2').id = 'CERS2p26er'
model.reactions.get_by_id('CERS2p26er').name = 'Ceramide 2p synthase  26C'
for m in r.metabolites:
    model.reactions.get_by_id('CERS2p26er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.reactions.get_by_id('CERS2p26er').add_metabolites({'cer2_26_r': -1.0, 'cer2p_26_r': 1.0})
m1 = model.metabolites.get_by_id('cer3_24_c').copy()
m2 = model.metabolites.get_by_id('cer3_26_c').copy()
m1.id = 'cer3_24_r'
m1.compartment = 'r'
m2.id = 'cer3_26_r'
m2.compartment = 'r'
model.add_metabolites([m1,m2])
r = model.reactions.get_by_id('CERS324').copy()
model.reactions.get_by_id('CERS324').id = 'CERS324er'
for m in r.metabolites:
    model.reactions.get_by_id('CERS324er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})

r = model.reactions.get_by_id('CERS326').copy()
model.reactions.get_by_id('CERS326').id = 'CERS326er'
for m in r.metabolites:
    model.reactions.get_by_id('CERS326er').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
m1 = model.metabolites.get_by_id('2hhxdal_c').copy()
m2 = model.metabolites.get_by_id('hxdcal_c').copy()
m1.id = '2hhxdal_r'
m1.compartment = 'r'
model.add_metabolites([m1])
r = model.reactions.get_by_id('PSPHPL').copy()
model.reactions.get_by_id('PSPHPL').id = 'PSPHPLer'
for m in r.metabolites:
    model.reactions.get_by_id('PSPHPLer').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
r = model.reactions.get_by_id('SPHPL').copy()
model.reactions.get_by_id('SPHPL').id = 'SPHPLer'
for m in r.metabolites:
    model.reactions.get_by_id('SPHPLer').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.remove_reactions(['SGPL11r','SGPL12r','SGPL13','yli_R1567','yli_R1568'], remove_orphans=True)

#### FMNAT

In [45]:
for r in sorted(model.genes.get_by_id('11542').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9298').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

AFAT atp_c + fmn_c + 2.0 h_c --> fad_c + ppi_c 11542 or 9298
FMNAT atp_c + fmn_c + h_c --> fad_c + ppi_c 11542
FMNATm atp_m + fmn_m + h_m --> fad_m + ppi_m 11542

AFAT atp_c + fmn_c + 2.0 h_c --> fad_c + ppi_c 11542 or 9298
RBFK atp_c + ribflv_c --> adp_c + fmn_c + h_c 9298
RBFKm atp_m + ribflv_m --> adp_m + fmn_m + h_m 9298


11542 cyto FAD1  
16092 mito FAD synthetase  
9298 mito 11, cyto_mito 10.666 FMN1 Riboflavin kinase

FMNAT has correct stoichiometry, AFAT incorrect  
Change FMNATm genes to '16092'
Remove AFAT

In [46]:
model.reactions.get_by_id('FMNATm').gene_reaction_rule = '16092'
model.remove_reactions(['AFAT'], remove_orphans=True)

#### ALDD2x

In [47]:
model.remove_reactions(['ALDD2x_copy1','GTPCI_2'], remove_orphans=True)

#### GTHOr

In [48]:
for r in sorted(model.genes.get_by_id('15482').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15038').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16549').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8790').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9250').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

GDR gthox_c + h_c + nadh_c --> 2.0 gthrd_c + nad_c 15482
GDR_nadp_h gthox_h + h_h + nadph_h --> 2.0 gthrd_h + nadp_h 15482
GDRh gthox_h + h_h + nadh_h --> 2.0 gthrd_h + nad_h 15482
GDRm gthox_m + h_m + nadh_m --> 2.0 gthrd_m + nad_m 15482
GTHOm gthox_m + h_m + nadph_m --> 2.0 gthrd_m + nadp_m 15482 or (15482 and 9250)
GTHOr gthox_c + h_c + nadph_c <=> 2.0 gthrd_c + nadp_c 15482 or (15038 and 15482) or (15482 and 16549) or (15482 and 8790)
TRDR h_c + nadph_c + trdox_c --> nadp_c + trdrd_c 15339 or 15482 or 16019 or 9688 or (CRv4_Au5_s2_g8777_t1 and CRv4_Au5_s9_g15314_t1) or (CRv4_Au5_s2_g8777_t1 and 15339) or (CRv4_Au5_s2_g8777_t1 and 16019) or (CRv4_Au5_s8_g14830_t1 and CRv4_Au5_s9_g15314_t1) or (CRv4_Au5_s8_g14830_t1 and 15339) or (CRv4_Au5_s8_g14830_t1 and 16019) or (CRv4_Au5_s9_g15314_t1 and 9688) or (15339 and 9688) or (16019 and 9688)
TRDRm h_m + nadph_m + trdox_m --> nadp_m + trdrd_m 15482 or (YCR083W and 9688) or (15339 and 9688)
yli_R0291 gthox_c + h_c + nadph_c --> gthrd_c + n

In [49]:
for r in sorted(model.metabolites.get_by_id('dhdascb_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('paps_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DASCBR dhdascb_c + h_c + nadph_c --> ascb__L_c + nadp_c 15038
DHAAt1r dhdascb_e <=> dhdascb_c 15762

ADSK aps_c + atp_c --> adp_c + h_c + paps_c 8709
BPNT2 h2o_c + paps_c --> aps_c + pi_c 10673
PAPSPAPthr pap_c + paps_h <=> pap_h + paps_c 10998 or 11267
PAPSR paps_c + trdrd_c --> 2.0 h_c + pap_c + so3_c + trdox_c 11741 or (11741 and 15339) or (11741 and 16019)
PAPSR2 grxrd_c + paps_c --> grxox_c + 2.0 h_c + pap_c + so3_c (b0849 and 11741) or (b1064 and 11741) or (11741 and 15038) or (11741 and 16549) or (11741 and 9250)
PAPStg paps_c <=> paps_g 10998 or 11267


Glutathione  
15482 cyto GLR1 cytosolic and mitochondrial glutathione oxidoreductase  
15038 nucl 10.5, cyto_nucl 9.333, mito 8.5, cyto_mito 8.333 GRX1 glutaredoxin  
8790 cyto_nucl 13.333, cyto 12.5, nucl 11, mito_nucl 6.999 GRX3,GRX4 glutaredoxin  
16549 mito (no sigP) GRX2 glutaredoxin  
9250 mito GRX5 glutaredoxin  
13734 mito (anchor, 416 aa) glutaredoxin  
8579 cyto 17, cyto_nucl 12.5 GPX2,HYR1

12715 cyto 15.5, cyto_mito 9.5 TSA1,TSA2 -> THIORDXi

Change GTHOr genes to '15482'  
Change GRXR genes to '15038 or 8790'  
Change GTHPi genes to '8579'  
Change GTHOm genes to '15482'   
Add GRXRm, and change genes to '13734 or 16549 or 9250'  
Remove GDR, GDR_nadp_h, GDRh, GDRm, yli_R0291, GTHPm

Replace DASCBR (nadph) with DHAOX_c (gthrd) from iLB1027_lipid  
PAPSR, 11741 cyto 13.5, cyto_nucl 12.333 MET16 uses thioredoxin

Thioredoxin reductase  
9688 mito 25.5, cyto_mito 14 (no sigP) TRR1,TRR2 -> cyto_mito  
9687 has sigP, but it seems truncated (~20% coverage), downstream of 9688, long intron could make another isoform    
Thioredoxins  
15339 mito 11, cyto_mito 9.166, pero 7, cyto_nucl 5.166, cyto 5 (no sigP) TRX1,TRX2 -> pero  
16019 mito 25 thioredoxin by orthomcl (TRX3?) -> mito  
10848 cyto 17.5, cyto_nucl 12.5 thioredoxin -> cyto_nucl  
12730 cyto 11.5, cyto_nucl 8.5 thioredoxin -> cyto_nucl  
12737 cyto 16, cyto_nucl 12 thioredoxin -> cyto_nucl

Change PAPSR genes to '(10848 and 11741) or (11741 and 12730) or (11741 and 12737) or (11741 and 15339)'  
Remove PAPSPAPthr, PAPSR2

In S. cer, cyto TRDR trx1, trx2, trr1 / mito TRDRm trx3, trr2
Change TRDR genes to '(10848 and 9688) or (12730 and 9688) or (12737 and 9688)'  
Change TRDRm genes to '16019 and 9688'  
RNDRs use thioredoxins  
Remove RNDR1b, RNDR2b, RNDR3b, RNDR3b

In [50]:
model.reactions.get_by_id('GTHOr').gene_reaction_rule = '15482'
model.reactions.get_by_id('GRXR').gene_reaction_rule = '15038 or 8790'
model.reactions.get_by_id('GTHPi').gene_reaction_rule = '8579'
model.reactions.get_by_id('GTHOm').gene_reaction_rule = '15482'
m1 = model.metabolites.get_by_id('grxox_c').copy()
m2 = model.metabolites.get_by_id('grxrd_c').copy()
m1.id = 'grxox_m'
m1.compartment = 'm'
m2.id = 'grxrd_m'
m2.compartment = 'm'
model.add_metabolites([m1,m2])
r = model.reactions.get_by_id('GRXR').copy()
r.id = 'GRXRm'
r.gene_reaction_rule = '13734 or 16549 or 9250'
model.add_reactions([r])
for m in model.reactions.get_by_id('GRXR').metabolites:
    model.reactions.get_by_id('GRXRm').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})
model.remove_reactions(['GDR','GDR_nadp_h','GDRh','GDRm','yli_R0291','GTHPm'], remove_orphans=True)
model.add_reactions([ptri.reactions.get_by_id('DHAOX_c').copy()])
model.reactions.get_by_id('DHAOX_c').gene_reaction_rule = '15038'
model.reactions.get_by_id('DASCBR').remove_from_model(remove_orphans=True)
model.reactions.get_by_id('PAPSR').gene_reaction_rule = '(10848 and 11741) or (11741 and 12730) or (11741 and 12737) or (11741 and 15339)'
model.reactions.get_by_id('PAPSPAPthr').remove_from_model(remove_orphans=True)
model.reactions.get_by_id('PAPSR2').remove_from_model(remove_orphans=True)
model.reactions.get_by_id('TRDR').gene_reaction_rule = '(10848 and 9688) or (12730 and 9688) or (12737 and 9688)'
model.reactions.get_by_id('TRDRm').gene_reaction_rule = '16019 and 9688'
model.remove_reactions(['RNDR1b','RNDR2b','RNDR3b','RNDR4b'], remove_orphans=True)

In [51]:
for r in sorted(model.genes.get_by_id('9688').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15339').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

TDSRh h_h + nadph_h + trdox_h --> nadp_h + trdrd_h (CRv4_Au5_s23_g9860_t1 and CRv4_Au5_s2_g8777_t1) or (CRv4_Au5_s23_g9860_t1 and CRv4_Au5_s8_g14830_t1) or (CRv4_Au5_s23_g9860_t1 and 9688) or (CRv4_Au5_s2_g8777_t1 and CRv4_Au5_s5_g12205_t1) or (CRv4_Au5_s2_g8777_t1 and 15339) or (CRv4_Au5_s2_g8777_t1 and 16019) or (CRv4_Au5_s5_g12205_t1 and CRv4_Au5_s8_g14830_t1) or (CRv4_Au5_s5_g12205_t1 and 9688) or (CRv4_Au5_s8_g14830_t1 and 15339) or (CRv4_Au5_s8_g14830_t1 and 16019) or (15339 and 9688) or (16019 and 9688)
TRDR h_c + nadph_c + trdox_c --> nadp_c + trdrd_c (10848 and 9688) or (12730 and 9688) or (12737 and 9688)
TRDRm h_m + nadph_m + trdox_m --> nadp_m + trdrd_m 16019 and 9688

AHAL achms_h + trdrd_h + tsul_h --> ac_h + h_h + hcys__L_h + so3_h + trdox_h (CRv4_Au5_s23_g9860_t1 and CRv4_Au5_s60_g12367_t1) or (CRv4_Au5_s5_g12205_t1 and CRv4_Au5_s60_g12367_t1) or (CRv4_Au5_s5_g12205_t2 and CRv4_Au5_s60_g12367_t1) or (CRv4_Au5_s60_g12367_t1 and 15339) or (CRv4_Au5_s60_g12367_t1 and 16019

Thioredoxin  
Remove TDSRh, CYSS_trdrd, AHAL -> check cysteine biosynthesis  
Remove DSBDR (E. coli specific)  
15902 mito 8, cysk 6, cyto_mito 6 MXR1 (cyto) methionine-S-sulfoxide reductase (peptide methionine, non-peptide)  
15469 cyto_mito (no sigP) YKL069W (cyto) methionine-R-sulfoxide reductase (non-peptide methionine)  
9153 mito MXR2 (mito) methionine-R-sulfoxide reductase (peptide methionine)  
Change METSOXR1 genes to '(10848 and 15902) or (12730 and 15902) or (12737 and 15902) or (15339 and 15902)'  
Change METSOXR2 genes to '(10848 and 15469) or (12730 and 15469) or (12737 and 15469) or (15339 and 15469)'

RNDR needs small heterodimer and large homodimer  
11290 cyto 14.5, cyto_mito 12.5 RNR1,RNR3 ribonucleotide reductase, alpha subunit (large)  
14237 cyto 13.5, cyto_nucl 10.5 RNR2 ribonucleotide reductase, beta subunit (small)  
11172 cyto_nucl RNR2 ribonucleotide reductase, beta subunit (small)  
Change RNDRx genes to '11172 and 11290 and 14237 and trdrd_cyto (10848 or 12730 or 12737 or 15339)'  
'(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237) or (11172 and 11290 and 14237 and 15339)'  
Change RNDRnx genes to '11172 and 11290 and 14237 and trdrd_nucl (10848 or 12730 or 12737)'  
'(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237)'  
Remove RNTRx (not present in S. cer and no genes in Rhodo)

Thioredoxin peroxidase  
beta-oxidation http://www.jbc.org/content/274/6/3402.long  
12715 cyto 15.5, cyto_mito 9.5 TSA1,TSA2 (cyto) -> THIORDXi  
15037 nucl 12.5, cyto_nucl 11 DOT5 (nucl) nuclear thiol peroxidase -> THIORDXni  
13262 cyto 20.5, cyto_nucl 12.5, pero 4 AHP1 (pero) -> THIORDXp  
ahp1-trx2 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3366830/  
10200 nucl 11, cyto 8, pero 5, cyto_mito 5 PRX1 (mito) -> THIORDXm  
Change THIORDXi genes to '(10848 and 12715) or (12715 and 12730) or (12715 and 12737)'  
Change THIORDXni genes to '(10848 and 15037) or (12730 and 15037) or (12737 and 15037)'  
Change THIORDXp genes to '13262 and 15339'
Change THIORDXm genes to '10200 and 16019'  

In [52]:
model.remove_reactions(['TDSRh','CYSS_trdrd','AHAL','DSBDR'], remove_orphans=True)
model.reactions.get_by_id('METSOXR1').gene_reaction_rule = '(10848 and 15902) or (12730 and 15902) or (12737 and 15902) or (15339 and 15902)'
model.reactions.get_by_id('METSOXR2').gene_reaction_rule = '(10848 and 15469) or (12730 and 15469) or (12737 and 15469) or (15339 and 15469)'
model.reactions.get_by_id('RNDR1').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237) or (11172 and 11290 and 14237 and 15339)'
model.reactions.get_by_id('RNDR2').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237) or (11172 and 11290 and 14237 and 15339)'
model.reactions.get_by_id('RNDR3').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237) or (11172 and 11290 and 14237 and 15339)'
model.reactions.get_by_id('RNDR4').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237) or (11172 and 11290 and 14237 and 15339)'
model.reactions.get_by_id('RNDR1n').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237)'
model.reactions.get_by_id('RNDR2n').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237)'
model.reactions.get_by_id('RNDR3n').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237)'
model.reactions.get_by_id('RNDR4n').gene_reaction_rule = '(10848 and 11172 and 11290 and 14237) or (11172 and 11290 and 12730 and 14237) or (11172 and 11290 and 12737 and 14237)'
model.remove_reactions(['RNTR1','RNTR2','RNTR3','RNTR4'], remove_orphans=True)
model.reactions.get_by_id('THIORDXi').gene_reaction_rule = '(10848 and 12715) or (12715 and 12730) or (12715 and 12737)'
model.reactions.get_by_id('THIORDXni').gene_reaction_rule = '(10848 and 15037) or (12730 and 15037) or (12737 and 15037)'
model.reactions.get_by_id('THIORDXp').gene_reaction_rule = '13262 and 15339'
model.reactions.get_by_id('THIORDXm').gene_reaction_rule = '10200 and 16019'

In [53]:
for r in sorted(model.genes.get_by_id('15679').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15496').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8943').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ATDGDm atp_m + dgdp_m + h_m --> adp_m + dgtp_m 15679
ATGDm atp_m + gdp_m + h_m --> adp_m + gtp_m 15679
NDPK1 atp_c + gdp_c <=> adp_c + gtp_c 15496 or 15679 or 8943
NDPK10 atp_c + didp_c <=> adp_c + ditp_c 15679 or 8943
NDPK10n atp_n + didp_n <=> adp_n + ditp_n 15679
NDPK1n atp_n + gdp_n <=> adp_n + gtp_n 15679
NDPK2 atp_c + udp_c <=> adp_c + utp_c 15496 or 15679 or 8943
NDPK2m atp_m + udp_m --> adp_m + utp_m 15679
NDPK2n atp_n + udp_n <=> adp_n + utp_n 15679
NDPK3 atp_c + cdp_c <=> adp_c + ctp_c 15496 or 15679 or 8943
NDPK3m atp_m + cdp_m --> adp_m + ctp_m 15679
NDPK3n atp_n + cdp_n <=> adp_n + ctp_n 15679
NDPK4 atp_c + dtdp_c <=> adp_c + dttp_c 15496 or 15679 or 8943
NDPK4m atp_m + dtdp_m --> adp_m + dttp_m 15679
NDPK4n atp_n + dtdp_n <=> adp_n + dttp_n 15679
NDPK5 atp_c + dgdp_c <=> adp_c + dgtp_c 15496 or 15679 or 8943
NDPK5n atp_n + dgdp_n <=> adp_n + dgtp_n 15679
NDPK6 atp_c + dudp_c <=> adp_c + dutp_c 15496 or 15679 or 8943
NDPK6m atp_m + dudp_m --> adp_m + dutp_m 15679
NDPK6n at

15679 cyto YNK1 (cyto, mito intermembrane space) nucleoside diphosphate kinase  
8943 mito nucleoside diphosphate kinase  

Remove NPDKxm, NDPKxn  
Add NPDKxm (reversible) from Recon1  
Change NDPKx genes to '15679'  
Change NDPKxm genes to '8943'

In [54]:
model.remove_reactions([r.id for r in model.reactions if r.id.startswith('NDPK') and r.id.endswith('n')], remove_orphans=True)
model.remove_reactions([r.id for r in model.reactions if r.id.startswith('NDPK') and r.id.endswith('m')], remove_orphans=True)
model.add_reactions([r.copy() for r in hsa.reactions if r.id.startswith('NDPK') and r.id.endswith('m')])
for r in model.reactions:
    if r.id.startswith('NDPK') and r.id.endswith('m'):
        r.gene_reaction_rule = '8943'
    elif r.id.startswith('NDPK'):
        r.gene_reaction_rule = '15679'

In [55]:
for r in sorted(model.genes.get_by_id('15496').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15129').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12300').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8385').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ADK1 amp_c + atp_c <=> 2.0 adp_c 12300 or 13190 or 15496
ADK1m amp_m + atp_m <=> 2.0 adp_m 12300 or 15129 or 15496
ADK3 amp_c + gtp_c <=> adp_c + gdp_c 15496
ADK4 amp_c + itp_c <=> adp_c + idp_c 15496
ADNK1 adn_c + atp_c --> adp_c + amp_c + h_c 15496 or 8385
DADK atp_c + damp_c <=> adp_c + dadp_c 12300 or 15129 or 15496

ADK1m amp_m + atp_m <=> 2.0 adp_m 12300 or 15129 or 15496
ADK3m amp_m + gtp_m <=> adp_m + gdp_m 15129
ADK4m amp_m + itp_m <=> adp_m + idp_m 15129
DADK atp_c + damp_c <=> adp_c + dadp_c 12300 or 15129 or 15496

ADK1 amp_c + atp_c <=> 2.0 adp_c 12300 or 13190 or 15496
ADK1m amp_m + atp_m <=> 2.0 adp_m 12300 or 15129 or 15496
ATAMh amp_h + atp_h --> 2.0 adp_h 12300
ATDAMh atp_h + damp_h --> adp_h + dadp_h 12300
ATDAMm atp_m + damp_m --> adp_m + dadp_m 12300
DADK atp_c + damp_c <=> adp_c + dadp_c 12300 or 15129 or 15496

ADNK1 adn_c + atp_c --> adp_c + amp_c + h_c 15496 or 8385
ADNK1m adn_m + atp_m --> adp_m + amp_m + h_m 8385


15496 cyto ADK1 adenylate kinase (also mito intermembrane space)
15129 mito ADK2 adenylate kinase  
12300 mito FAP7 Essential NTPase required for small ribosome subunit synthesis  
8385 extr (sigP) ADO1 adenosine kinase (cyto and nucl?)  

S. cer ADK1 catalyze ADK1 and DADK, and ADK2 catalyze and ADK3m and ADK4m  
http://www.jbc.org/content/280/19/18604.full  
Change ADK1 genes to '15496'  
Change DADK genes to '15496'  
Change ADK3m genes to '15129'  
Remove ADK1m, ADK3, ADK4, ATAMh, ATDAMh, ATDAMm

Change ADNK1 genes to '8385'
Remove ADNK1m

In [56]:
model.reactions.get_by_id('ADK1').gene_reaction_rule = '15496'
model.reactions.get_by_id('DADK').gene_reaction_rule = '15496'
model.reactions.get_by_id('ADK3m').gene_reaction_rule = '15129'
model.reactions.get_by_id('ADK4m').gene_reaction_rule = '15129'
model.remove_reactions(['ADK1m','ADK3','ADK4','ATAMh','ATDAMh','ATDAMm'], remove_orphans=True)
model.reactions.get_by_id('ADNK1').gene_reaction_rule = '8385'
model.remove_reactions(['ADNK1m'], remove_orphans=True)

In [57]:
for r in sorted(model.genes.get_by_id('15252').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13190').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DTMPK atp_c + dtmp_c <=> adp_c + dtdp_c 15252
NDP8 dudp_c + h2o_c --> dump_c + h_c + pi_c 15252
URIDK2r atp_c + dump_c <=> adp_c + dudp_c 13190 or 15252

CPK1 cmp_c + ctp_c <=> 2.0 cdp_c 13190
CYTK1 atp_c + cmp_c <=> adp_c + cdp_c 13190
CYTK10 cmp_c + dgtp_c <=> cdp_c + dgdp_c 13190
CYTK10n cmp_n + dgtp_n <=> cdp_n + dgdp_n 13190
CYTK11 dcmp_c + dgtp_c <=> dcdp_c + dgdp_c 13190
CYTK11n dcmp_n + dgtp_n <=> dcdp_n + dgdp_n 13190
CYTK12 dcmp_c + dctp_c <=> 2.0 dcdp_c 13190
CYTK12n dcmp_n + dctp_n <=> 2.0 dcdp_n 13190
CYTK13 datp_c + dcmp_c <=> dadp_c + dcdp_c 13190
CYTK13n datp_n + dcmp_n <=> dadp_n + dcdp_n 13190
CYTK14 dcmp_c + utp_c <=> dcdp_c + udp_c 13190
CYTK14n dcmp_n + utp_n <=> dcdp_n + udp_n 13190
CYTK1n atp_n + cmp_n <=> adp_n + cdp_n 13190
CYTK2 atp_c + dcmp_c <=> adp_c + dcdp_c 13190
CYTK2_1 ctp_c + dcmp_c <=> cdp_c + dcdp_c 13190
CYTK2n atp_n + dcmp_n <=> adp_n + dcdp_n 13190
CYTK3n ctp_n + dcmp_n <=> cdp_n + dcdp_n 13190
CYTK4n dcmp_n + gtp_n <=> dcdp_n + gdp_n 13190
CYTK5n

15252 mito (no sigP) CDC8 thymidylate kinase (nucl and cyto) -> DTMPK  
13190 cysk 10, cyto 8, mito 7 (no sigP) URA6 Uridylate kinase/adenylate kinase (cyto predominantly and nucl) / KEGG UMP-CMP kinase  
URA6 activity controversy -> keep these reactions for now

Change URIDK2r genes to '13190'

In [58]:
model.reactions.get_by_id('URIDK2r').gene_reaction_rule = '13190'

In [59]:
for r in sorted(model.metabolites.get_by_id('hom__L_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('achms_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('hcys__L_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('cyst__L_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16618').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule) 

HSDxi aspsa_c + h_c + nadh_c --> hom__L_c + nad_c 12080
HSDy hom__L_c + nadp_c <=> aspsa_c + h_c + nadph_c 12080 or 16738
HSERTA accoa_c + hom__L_c <=> achms_c + coa_c 12513 or 15248 or (PP_5098 and 15248)
HSK atp_c + hom__L_c --> adp_c + h_c + phom_c 8651

AHSERL achms_c + ch4s_c --> ac_c + h_c + met__L_c 16618
AHSERL2 achms_c + h2s_c --> ac_c + h_c + hcys__L_c 16618
HSERTA accoa_c + hom__L_c <=> achms_c + coa_c 12513 or 15248 or (PP_5098 and 15248)
METB1 achms_c + cys__L_c --> ac_c + cyst__L_c + h_c 11463 or 16725 or 16742
yli_R0094 achms_c + h_c + trdrd_c + tsul_c --> ac_c + hcys__L_c + so3_c + trdox_c 11463 or 16725 or 16742

AHCi ahcys_c + h2o_c --> adn_c + hcys__L_c 12912
AHSERL2 achms_c + h2s_c --> ac_c + h_c + hcys__L_c 16618
CYSTL cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c 8759
CYSTS hcys__L_c + ser__L_c --> cyst__L_c + h2o_c 15712
HCYSMT amet_c + hcys__L_c --> ahcys_c + h_c + met__L_c 15759
HCYSMT2 hcys__L_c + mmet_c --> h_c + 2.0 met__L_c 15759
METS 5mthf_c + hcys__L_c 

Cysteine biosynthesis  

12080 extr HOM6 (cyto) homoserine dehydrogenase  
16738 cyto homoserine dehydrogenase  
8651 mito THR1 homoserine kinase

12513 cyto_nucl MET2 L-homoserine O-acetyltransferase  
15248 nucl MET2 L-homoserine O-acetyltransferase  
16618 cyto MET17 O-acetylhomoserine sulfhydrylase  
15712 cyto CYS4 cystathionine beta-synthase  
9499 mito CYS3 cystathionine gamma-lyase  
8759 mito (no sigP) STR3 (pero) cystathionine beta-lyase (also involved in 3-mercaptohexanol)  
11463 mito (no sigP) STR2 (cyto),YLL058W,YML082W (cyto) cystathionine gamma-synthase
16725 cyto YHR112C cystathionine beta-lyase/gamma-synthase, unknown function in S. cer  
16742 plas YHR112C cystathionine beta-lyase/gamma-synthase, unknown function in S. cer  

serine O-acetyltransferase is missing in Rhodo, 9734 is maltose O-acetyltransferase  
12031 mito (no sigP) MCY1 cysteine synthase A (O-acetyl-L-serine sulfhydrylase)  
13106 cyto_mito (anchor) MCY1 cysteine synthase A (O-acetyl-L-serine sulfhydrylase)

12912 cyto SAH1 S-adenosyl-L-homocysteine hydrolase  
15759 extr MHT1,SAM4,YMR321C S-methylmethionine-homocysteine methyltransferase  
9825 mito (non-sigP) MET6 cobalamin-independent methionine synthase  
12876 cyto cobalamin-independent methionine synthase  
12920 cyto cobalamin-independent methionine synthase

Change HSDxi genes to '12080 or 16738'  
Change HSERTA genes to '12513 or 15248'  
AHSERL, AHSERL2 is correct  
Remove AHSERL4, METB1, yli_R0094, SHSL2r (P. putida specific)  
Change METS genes to '12876 or 12920 or 9825'  
MHPGLUT is a specific 5mthf (mhpglu_c) blocked rxn  
Remove MHPGLUT  
Change CYSTL genes to '16725 or 16742 or 8759'  
Remove CYSTLp  
CHange SHSL1 genes to '11463'

In [60]:
model.reactions.get_by_id('HSDxi').gene_reaction_rule = '12080 or 16738'
model.reactions.get_by_id('HSERTA').gene_reaction_rule = '12513 or 15248'
model.remove_reactions(['AHSERL4','METB1','yli_R0094','SHSL2r'], remove_orphans=True)
model.reactions.get_by_id('METS').gene_reaction_rule = '12876 or 12920 or 9825'
model.remove_reactions(['MHPGLUT'], remove_orphans=True)
model.reactions.get_by_id('CYSTL').gene_reaction_rule = '16725 or 16742 or 8759'
model.remove_reactions(['CYSTLp'], remove_orphans=True)
model.reactions.get_by_id('SHSL1').gene_reaction_rule = '11463'

In [61]:
temp = ['12080','14662','16738','8651','12513','15248','16618','15712','9499','8759','11463','16725','16742',
        '9734','12031','13106','12912','15759','9825','12876','12920']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12080,K00003: E1.1.1.3; homoserine dehydrogenase,S,HOM6,,HOM6,,Essential,"extr 10, cyto 7.5, mito 7, cyto_nucl 4.5",VHL*
14662,K00928: lysC; aspartate kinase,,HOM3,,HOM3,,Essential,"cyto 13, mito 12",VFS*
16738,K12525: metL; bifunctional aspartokinase / hom...,,,,HOM6,,Not Essential,"cyto 13, cyto_mito 8.833, extr 8, cyto_nucl 7....",VGL*
8651,K00872: thrB1; homoserine kinase,,THR1,,THR1,,Essential,"mito 14, cyto 7.5, cyto_nucl 4.5, extr 2, pero 2",EHK*
12513,K00641: metX; homoserine O-acetyltransferase,,MET2,,MET2,,Not Essential,"cyto_nucl 12, nucl 11.5, cyto 11.5, pero 2",TRW*
15248,K00641: metX; homoserine O-acetyltransferase,,,,MET2,,Not Essential,"nucl 20, cyto 4, mito 2",ADP*
16618,K17069: MET17; O-acetylhomoserine/O-acetylseri...,,MET17,,MET17,CTH,Not Essential,"cyto 19, cyto_nucl 12.5, pero 3",TAL*
15712,"K01697: E4.2.1.22, CBS; cystathionine beta-syn...",,CYS4,"CBS,CBSL",CYS4,CBSL,Not Essential,"cyto 20.5, cyto_mito 13, mito 4.5",GDA*
9499,K01758: CTH; cystathionine gamma-lyase,,CYS3,CTH,CYS3,CTH,Not Essential,"mito 21, cyto 6",VKA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16738,0.558274,0.307275,0.626728,0.565536,0.507002,0.365426,0.298877,-0.137587,0.672879,0.366337,-0.169222
12513,-0.835478,-1.0497,-1.03052,0.326171,-0.144332,0.0229116,-3.87828,-4.15135,-5.17453,-0.357745,-0.323942
15248,0.0989456,-0.365113,0.00543596,0.136312,-0.432678,-1.73711,-0.382425,-1.90233,-2.30563,0.256682,0.310651
16618,0.0419636,0.0304368,-0.143477,-0.158729,-0.0705999,-0.052245,0.209139,0.397923,0.195869,0.231633,0.0845513
15712,-0.231243,-0.613382,-0.146129,-0.19365,0.19931,-0.126857,0.648164,-0.66015,-0.333046,-0.497314,-0.0583468
9499,-0.0362174,-0.108912,-0.420679,-0.254394,0.149364,0.116627,-0.365395,-0.485208,-0.0270825,0.108624,0.206243
11463,-0.167618,-0.0282166,-0.0612006,-0.182825,0.106678,-0.0196322,0.0520783,-0.172054,0.0643711,-0.14793,0.0628106
16725,-0.0611906,-0.56125,0.0652367,-0.119765,-0.245465,-0.155754,-0.148042,-0.632691,0.447575,0.495966,0.353243
16742,-0.143095,0.0413233,-0.0158122,0.116166,0.384373,0.0921014,-0.0265362,-0.240916,-0.377589,-0.582753,-0.481596


In [62]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ASPK: asp__L_c + atp_c <=> 4pasp_c + adp_c 12080 or 14662 or 16738
ASPK_1: asp__L_h + atp_h --> 4pasp_h + adp_h 12080 or 14662
HSDH: aspsa_h + h_h + nadph_h --> hom__L_h + nadp_h 12080
HSDxi: aspsa_c + h_c + nadh_c --> hom__L_c + nad_c 12080 or 16738
HSDy: hom__L_c + nadp_c <=> aspsa_c + h_c + nadph_c 12080 or 16738

ASPK: asp__L_c + atp_c <=> 4pasp_c + adp_c 12080 or 14662 or 16738
ASPK_1: asp__L_h + atp_h --> 4pasp_h + adp_h 12080 or 14662

ASPK: asp__L_c + atp_c <=> 4pasp_c + adp_c 12080 or 14662 or 16738
HSDxi: aspsa_c + h_c + nadh_c --> hom__L_c + nad_c 12080 or 16738
HSDy: hom__L_c + nadp_c <=> aspsa_c + h_c + nadph_c 12080 or 16738

HSK: atp_c + hom__L_c --> adp_c + h_c + phom_c 8651

HSERTA: accoa_c + hom__L_c <=> achms_c + coa_c 12513 or 15248

HSERTA: accoa_c + hom__L_c <=> achms_c + coa_c 12513 or 15248

AHSERL: achms_c + ch4s_c --> ac_c + h_c + met__L_c 16618
AHSERL2: achms_c + h2s_c --> ac_c + h_c + hcys__L_c 16618

ACSERL: acser_c + seln_c <=> ac_c + 2.0 h_c + selcys_c 12

In [63]:
# 14662 aspartate kinase
model.reactions.get_by_id('ASPK').gene_reaction_rule = '14662'
model.remove_reactions(['ASPK_1','HSDH'], remove_orphans=True)
# 15712 CYS4 cystathionine beta-synthase (4.2.1.22)
# L-Serine + L-Homocysteine <=> L-Cystathionine + H2O (CYSTS, correct)
# 12031 and 13106 cysteine synthase A / O-acetyl-L-serine sulfhydrylase (2.5.1.47)
# O-Acetyl-L-serine + Hydrogen sulfide <=> L-Cysteine + Acetate (CYSS)
model.reactions.get_by_id('CYSS').gene_reaction_rule = '12031 or 13106'
# ACSERL/SLCYSS is by selenocysteine synthase, and ACSERSULL by cysteine synthase B only
# CHOLS_ex and CYSS_1 wrong compartment, CYS wrong, SELCYSTS not known
model.remove_reactions(['ACSERL','ACSERLh','ACSERLm','SLCYSS','ACSERSULL','ACSERSULLh','ACSERSULLm',
                        'CHOLS_ex','CYSS_1','CYS','SELCYSTS'], remove_orphans=True)
# 9499 CYS3 cystathionine gamma-lyase -> CYSTGL
model.reactions.get_by_id('CYSTGL').gene_reaction_rule = '9499'
# 8759 STR3 cystathionine beta-lyase -> CYSTL
model.reactions.get_by_id('CYSTL').gene_reaction_rule = '8759'
# 11463 STR2 cystathionine gamma-synthase -> SHSL1
# SHSLr4 is sum of CYSTL and CYSTGL
model.remove_reactions(['SELCYSTGL','SHSL4r','CBL','CTINBL','CYSTBL','SELCYSTL','SELCYSTLh'], remove_orphans=True)
# 16725 and 16742 best hit is A. fum cysteine-S-conjugate beta-lyase -> CYSDS
model.reactions.get_by_id('CYSDS').gene_reaction_rule = '16725 or 16742'
# Remove the rest of seleno-reactions
# MS incorrect proton, mhpglu is correct in biocyc but MS is the only reaction using this
model.remove_reactions(['SEAHCYSHYD','SEAHCYSHYD_1','MS'], remove_orphans=True)

In [64]:
for r in sorted(model.metabolites.get_by_id('cys__L_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

AMPTASECG: cgly_c + h2o_c --> cys__L_c + gly_c 10210 or 12096
CYSDS: cys__L_c + h2o_c --> h2s_c + nh4_c + pyr_c 16725 or 16742
CYSS: acser_c + h2s_c --> ac_c + cys__L_c + h_c 12031 or 13106
CYSTA: akg_c + cys__L_c --> glu__L_c + mercppyr_c 14281 or 8936
CYSTGL: cyst__L_c + h2o_c --> 2obut_c + cys__L_c + nh4_c 9499
CYSTRS: atp_c + cys__L_c + trnacys_c --> amp_c + cystrna_c + ppi_c 9855
CYSt2r: cys__L_e + h_e <=> cys__L_c + h_c 14229 or 15074
GLUCYS: atp_c + cys__L_c + glu__L_c --> adp_c + glucys_c + h_c + pi_c 12007 or 12022 or (GCLM and 12007) or (GCLM and 12022) or (Gclm and 12007) or (Gclm and 12022)
ICYSDS: cys__L_c + iscs_c --> ala__L_c + iscssh_c 13740
PPNCL: 4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cdp_c + h_c + pi_c 8536
PPNCL2: 4ppan_c + ctp_c + cys__L_c --> 4ppcys_c + cmp_c + h_c + ppi_c 8536 or 8878
PPNCL3: 4ppan_c + atp_c + cys__L_c --> 4ppcys_c + amp_c + h_c + ppi_c 8878
SHSL1: cys__L_c + suchms_c --> cyst__L_c + h_c + succ_c 11463


In [65]:
# 12007 ~100% coverage, but 12002 only ~50% coverage
model.reactions.get_by_id('GLUCYS').gene_reaction_rule = '12007'

#### UREASE

In [66]:
for r in sorted(model.genes.get_by_id('9326').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ALPHNH allphn_c + h2o_c + 3.0 h_c --> 2.0 co2_c + 2.0 nh4_c 9326
ALPHm allphn_m + h2o_m + 3.0 h_m --> 2.0 co2_m + 2.0 nh4_m 9326
URCB atp_c + hco3_c + urea_c --> adp_c + allphn_c + 2.0 h_c + pi_c 9326
URCBm atp_m + hco3_m + urea_m --> adp_m + allphn_m + 2.0 h_m + pi_m 9326
UREASE atp_c + hco3_c + urea_c <=> adp_c + allphn_c + h_c + pi_c 9326


9326 cyto 13.5, cyto_mito 13 (no sigP) DUR1,2 urea amidolyase  
UREASE is correct, URCB is incorrect
Remove ALPHm, URCB, URCBm

In [67]:
model.remove_reactions(['ALPHm','URCB','URCBm'], remove_orphans=True)

#### PTPATi

In [68]:
for r in sorted(model.genes.get_by_id('14849').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11114').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

APPAT atp_c + 2.0 h_c + pan4p_c <=> dpcoa_c + ppi_c 14849
DPCOAK atp_c + dpcoa_c --> adp_c + coa_c + h_c 11114 or 14849
PTPATi atp_c + h_c + pan4p_c --> dpcoa_c + ppi_c 14849
PTPATim atp_m + h_m + pan4p_m --> dpcoa_m + ppi_m 14849

DPCOAK atp_c + dpcoa_c --> adp_c + coa_c + h_c 11114 or 14849
DPCOAKm atp_m + dpcoa_m --> adp_m + coa_m + h_m 11114


14849 mito 13, cyto 10 (sigP) CAB4 pantetheine-phosphate adenylyltransferase  
11114 cyto (no sigP) CAB5 dephospho-CoA kinase  
PTPATi is correct, APPAT is incorrect, mito reactions are orphans  
Change DPCOAK genes to '11114'  
Remove APPAT, PTPATim, DPCOAKm

In [69]:
model.reactions.get_by_id('DPCOAK').gene_reaction_rule = '11114'
model.remove_reactions(['APPAT','PTPATim','DPCOAKm'], remove_orphans=True)

#### ACACT1x

In [70]:
for r in sorted(model.genes.get_by_id('8678').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8885').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACACT10m 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 13813 or 8678
ACACT1m 2.0 accoa_m --> aacoa_m + coa_m 8678 or 8885 or (HADHA and HADHB) or (Hadha and Hadhb)
ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c 13813 or 8678 or 8885
ACACT1x 2.0 accoa_x <=> aacoa_x + coa_x 8678 or 8885
ACACT2 3ohcoa_x + coa_x <=> accoa_x + btcoa_x 13813 or 8678 or 8885
ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c 13813 or 8678 or 8885
KAT1 aacoa_c + coa_c --> 2.0 accoa_c 13813 or 8678 or 8885
yli_R1435 2.0 accoa_x --> aacoa_x + coa_x 8678 or 8885

ACACT1m 2.0 accoa_m --> aacoa_m + coa_m 8678 or 8885 or (HADHA and HADHB) or (Hadha and Hadhb)
ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c 13813 or 8678 or 8885
ACACT1x 2.0 accoa_x <=> aacoa_x + coa_x 8678 or 8885
ACACT2 3ohcoa_x + coa_x <=> accoa_x + btcoa_x 13813 or 8678 or 8885
ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c 13813 or 8678 or 8885
KAT1 aacoa_c + coa_c --> 2.0 accoa_c 13813 or 8678 or 8885
yli_R1435 2.0 accoa_x --> aacoa_x + coa_x 8678 or 8885


In [71]:
# Fix simple things now and curate fatty acid metabolism later
model.reactions.get_by_id('ACACT1m').gene_reaction_rule = '8678 or 8885'
model.remove_reactions(['KAT1','yli_R1435'], remove_orphans=True)

In [72]:
#### Rest of duplicated

In [73]:
duplicated = set()
for i, r in enumerate(model.reactions):
    temp = 0
    for j, r2 in enumerate(model.reactions):
        if j > i and r2.id not in duplicated:
            if r.reactants == r2.reactants and r.products == r2.products:
                if temp == 0:
                    temp = 1
                    duplicated.add(r.id)
                    print(r.id, r.reaction, r.gene_reaction_rule)
                duplicated.add(r2.id)
                print(r2.id, r2.reaction, r2.gene_reaction_rule)
    if temp == 1:
        print()

yli_R0034 chtn_c + h2o_c --> acgam_c 13082
CHTNASE chtn_c + 2.0 h2o_c --> 3.0 acgam_c 13082

ARGSS asp__L_c + atp_c + citr__L_c --> amp_c + argsuc_c + h_c + ppi_c 16196
ARGSS_1 asp__L_c + atp_c + citr__L_c --> amp_c + argsuc_c + 2.0 h_c + ppi_c 16196

PRAGSr atp_c + gly_c + pram_c <=> adp_c + gar_c + h_c + pi_c 14259
PPRGL atp_c + gly_c + pram_c --> adp_c + gar_c + 2.0 h_c + pi_c 14259

AIRCr air_c + co2_c <=> 5aizc_c + h_c 12132
PRAIC air_c + co2_c <=> 5aizc_c + 2.0 h_c 12132

yli_R0224 8.0 coa_m + 8.0 h2o_m + 8.0 nad_m + nadph_m + 7.0 o2_m + yli_M04625_m --> 9.0 accoa_m + 7.0 h2o2_m + 7.0 h_m + 8.0 nadh_m + nadp_m (12742 and 13813 and 14805) or (12742 and 14805 and 9065) or (12752 and 13813 and 14805) or (12752 and 14805 and 9065) or (13813 and 14805 and 9700) or (14805 and 9065 and 9700)
yli_R0223 8.0 coa_m + 8.0 h2o_m + 8.0 nad_m + 2.0 nadph_m + 8.0 o2_m + yli_M04625_m --> 9.0 accoa_m + 8.0 h2o2_m + 6.0 h_m + 8.0 nadh_m + 2.0 nadp_m (12742 and 13813 and 14805) or (12742 and 14805 a

In [74]:
for x in ['13082','16196','14259','12132','10430','14638','16323','11646','9050','9708',
          '10475','12434','15385','11993','14256','10722','9434','9435','10104','14119','15736','16202']:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

CHTNASE chtn_c + 2.0 h2o_c --> 3.0 acgam_c 13082
CHTNASEe chtn_e + 2.0 h2o_e --> 3.0 acgam_e 13082
yli_R0034 chtn_c + h2o_c --> acgam_c 13082

ARGSS asp__L_c + atp_c + citr__L_c --> amp_c + argsuc_c + h_c + ppi_c 16196
ARGSS_1 asp__L_c + atp_c + citr__L_c --> amp_c + argsuc_c + 2.0 h_c + ppi_c 16196

FGFTh fgam_h + 3.0 h_h + thf_h --> gar_h + h2o_h + methf_h 13595 or 14259
FPGFTh 10fthf_h + gar_h <=> fgam_h + h_h + thf_h 13595 or 14259
GARFT 10fthf_c + gar_c <=> fgam_c + h_c + thf_c 13595 or 14259
PPRGL atp_c + gly_c + pram_c --> adp_c + gar_c + 2.0 h_c + pi_c 14259
PPRGLh atp_h + gly_h + pram_h --> adp_h + gar_h + h_h + pi_h 14259
PRAGSr atp_c + gly_c + pram_c <=> adp_c + gar_c + h_c + pi_c 14259
PRAIS atp_c + fpram_c --> adp_c + air_c + 2.0 h_c + pi_c 14259

AIRC2 air_c + atp_c + hco3_c --> 5caiz_c + adp_c + h_c + pi_c 12132
AIRC3 5aizc_c <=> 5caiz_c 12132
AIRCr air_c + co2_c <=> 5aizc_c + h_c 12132
PRAIC air_c + co2_c <=> 5aizc_c + 2.0 h_c 12132
PRAICh air_h + co2_h <=> 5aizc_h + h_

13082	extr 23, golg 2	K01183: E3.2.1.14; chitinase	GMQ*  
ARGSS_1 incorrect stoich  
PRAGSr, PRAIS correct (air_c -2 in BiGG, but -1 in Metacyc)  
AIRCr correct (AIRC2+AIRC3 bacteria, PRAIC with air_c -1)  

NMA1,NMA2	10430	cyto 13.5, cyto_mito 10.5, nucl 7, mito 6.5	K06210: NMNAT; nicotinamide mononucleotide adenylyltransferase	SAS*  
NPP1,NPP2	14638	cyto 8.5, extr 7, mito 6, cyto_nucl 5, plas 2	KOG2645: Type I phosphodiesterase/nucleotide pyrophosphatase	EGV*  
15385	cyto_mito 7, mito 6.5, cyto 6.5, pero 5, nucl 4, extr 2, cysk 2	K03574: mutT, NUDT15, MTH2; 8-oxo-dGTP diphosphatase	LVL*  

NMNAT/NMNATm/NMNATn, NNATr/NNATm/NNATn correct  
Change NNATr to NNAT, and make it irreversible (deltaG -54 kcal/mol in MetaCyc)  

GLYALDDr is not clear -> remove for now  

12434	mito 13.5, cyto_mito 10.833, cyto 7, cyto_nucl 5.333, pero 3	K03426: E3.6.1.22, NUDT12, nudC; NAD+ diphosphatase	SKM*  

14119	plas 22, E.R. 4	K14708: SLC26A11; solute carrier family 26 (sodium-independent sulfate anion transporter), member 11	TKA*  
15736	plas 25	K14708: SLC26A11; solute carrier family 26 (sodium-independent sulfate anion transporter), member 11	DDW*  
16682	plas 22, E.R. 3	K03321: TC.SULP; sulfate permease, SulP family	ASL*

In [75]:
temp = ['yli_R0034','ARGSS_1','FGFTh','FPGFTh','PPRGL','PPRGLh','AIRC2','AIRC3','PRAIC','PRAICh','ANNAT','ANNATn',
        'NNATr_copy1','NNATr_copy2','GLYALDDr','yli_R0303','yli_R1508','HDH','HDHh','PRACHh','PRADPh','GUAD_1','PBAL',
        'PBALm','NADDP','yli_R1538','yli_R1539','NPH','DNTPPA_1','STARCH300DEGR2A','STARCH300DEGR2B','STARCH300DEGRA',
        'STARCH300DEGRB','HDC','yli_R0777','CHOLSabc']
model.remove_reactions(temp, remove_orphans=True)

model.reactions.get_by_id('NNATr').id = 'NNAT'
model.reactions.get_by_id('NNAT').lower_bound = 0.0

r = sce.reactions.get_by_id('DNTPPA').copy()
r.gene_reaction_rule = '15385'
model.add_reactions([r])
r = hsa.reactions.get_by_id('3SALACBOXL').copy() # why is this not picked up? check
r.gene_reaction_rule = '10722 or 9434 or 9435'
model.add_reactions([r])

model.reactions.get_by_id('SO4ti').gene_reaction_rule = '14119 or 15736 or 16682'

#### Fix fatty acid and sterol in the next step

In [76]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
model

1332
3264
3290


Name,R. toruloides
Memory address,0x01027435ac8
Number of metabolites,3290
Number of reactions,3264
Number of groups,0
Objective expression,0
Compartments,"c, x, m, e, r, v, n, g, p, h, s, f, l"


In [77]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

12022
15595
4833_AT1
9153
CRv4_Au5_s6_g12448_t1
GCLM
Gclm
PDHX
PHATRDRAFT_34976
PHATRDRAFT_36641
PHATRDRAFT_37658
PHATRDRAFT_46880
PP_1986
PP_5098
Pdhx
YCR083W
YDR453C
YGR180C
b0071
b3551

2hhxdal_c
amob_c
dann_c
hxdcal_c
psph1p_c
psphings_c
sph1p_c


In [78]:
cobra.manipulation.remove_genes(model, [x for x in model.genes if not x.reactions])
model.remove_metabolites([x for x in model.metabolites if not x.reactions])

In [79]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
model

1312
3264
3283


Name,R. toruloides
Memory address,0x01027435ac8
Number of metabolites,3283
Number of reactions,3264
Number of groups,0
Objective expression,0
Compartments,"c, x, m, e, r, v, n, g, p, h, s, f, l"


In [80]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

In [81]:
cobra.io.save_json_model(model, "IFO0880_GPR_1b.json")

In [82]:
model_old = cobra.io.load_json_model("IFO0880_GPR_1a.json")
model_new = cobra.io.load_json_model("IFO0880_GPR_1b.json")

In [83]:
print('Removed reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r not in model_new.reactions:
        print(r)

Removed reactions

3DSPHR: 3dsphgn_c + h_c + nadph_c --> nadp_c + sphgn_c
4HTHRA: 4hthr_c <=> gcald_c + gly_c
4HTHRK: 4hthr_c + atp_c --> adp_c + h_c + phthr_c
4HTHRS: h2o_c + phthr_c --> 4hthr_c + pi_c
ACAS_2ahbut: 2ahethmpp_h + 2obut_h --> 2ahbut_h + thmpp_h
ACHBS: 2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c
ACLS: h_c + 2.0 pyr_c --> alac__S_c + co2_c
ACSERL: acser_c + seln_c <=> ac_c + 2.0 h_c + selcys_c
ACSERLh: acser_h + seln_h <=> ac_h + 2.0 h_h + selcys_h
ACSERLm: acser_m + seln_m <=> ac_m + 2.0 h_m + selcys_m
ACSERSULL: acser_c + tsul_c --> ac_c + h_c + scys__L_c
ACSERSULLh: acser_h + tsul_h --> ac_h + h_h + scys__L_h
ACSERSULLm: acser_m + tsul_m --> ac_m + h_m + scys__L_m
ADK1m: amp_m + atp_m <=> 2.0 adp_m
ADK3: amp_c + gtp_c <=> adp_c + gdp_c
ADK4: amp_c + itp_c <=> adp_c + idp_c
ADNK1m: adn_m + atp_m --> adp_m + amp_m + h_m
AFAT: atp_c + fmn_c + 2.0 h_c --> fad_c + ppi_c
AHAL: achms_h + trdrd_h + tsul_h --> ac_h + h_h + hcys__L_h + so3_h + trdox_h
AHSERL4: acser_c + trdrd_c +

In [84]:
print('Updated reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r in model_new.reactions:
        r2 = model_new.reactions.get_by_id(r.id)
        if (r.name == r2.name and r.reaction == r2.reaction and r.gene_reaction_rule == r2.gene_reaction_rule and
            r.lower_bound == r2.lower_bound and r.upper_bound == r2.upper_bound):
            pass
        else:
            print('Old', r, r.gene_reaction_rule)
            print('New', r2, r2.gene_reaction_rule)
            print()

Updated reactions

Old 2OXOADOXm: 2oxoadp_m + coa_m + nad_m --> co2_m + glutcoa_m + nadh_m (PDHX and 10007 and 10040 and 12116) or (PDHX and 10040 and 12116 and 9274) or (Pdhx and 10007 and 10040 and 12116) or (Pdhx and 10040 and 12116 and 9274)
New 2OXOADOXm: 2oxoadp_m + coa_m + nad_m --> co2_m + glutcoa_m + nadh_m 10040 and 12116 and 9274

Old ACACT1m: 2.0 accoa_m --> aacoa_m + coa_m 8678 or 8885 or (HADHA and HADHB) or (Hadha and Hadhb)
New ACACT1m: 2.0 accoa_m --> aacoa_m + coa_m 8678 or 8885

Old ADK1: amp_c + atp_c <=> 2.0 adp_c 12300 or 13190 or 15496
New ADK1: amp_c + atp_c <=> 2.0 adp_c 15496

Old ADNK1: adn_c + atp_c --> adp_c + amp_c + h_c 15496 or 8385
New ADNK1: adn_c + atp_c --> adp_c + amp_c + h_c 8385

Old AKGDm: akg_m + coa_m + nad_m --> co2_m + nadh_m + succoa_m (PDHX and 10007 and 10040 and 12116) or (PDHX and 10040 and 12116 and 9274) or (Pdhx and 10007 and 10040 and 12116) or (Pdhx and 10040 and 12116 and 9274)
New AKGDm: akg_m + coa_m + nad_m --> co2_m + nadh_m + 

In [85]:
print('Added reactions\n')
for r in sorted(model_new.reactions, key=lambda x: x.id):
    if r not in model_old.reactions:
        print(r)

Added reactions

2OH3K5MPPISO: h2o_c + hkmpp_c --> dhmtp_c + pi_c
3DSPHRer: 3dsphgn_r + h_r + nadph_r --> nadp_r + sphgn_r
3SALACBOXL: 3sala_c + h_c --> co2_c + hyptaur_c
ACRS: dkmpp_c --> h_c + hkmpp_c
AMAOTrm: 8aonn_m + amet_m <=> amob_m + dann_m
AOXSp: ala__L_x + h_x + pimcoa_x --> 8aonn_x + co2_x + coa_x
BTS5m: 2fe2s_m + amet_m + dtbt_m --> 2fe1s_m + btn_m + dad_5_m + h_m + met__L_m
CERH124er: cer1_24_r + h_r + nadph_r + o2_r --> cer2_24_r + h2o_r + nadp_r
CERH126er: cer1_26_r + h_r + nadph_r + o2_r --> cer2_26_r + h2o_r + nadp_r
CERS2p24er: cer1_24_r + h_r + nadph_r + o2_r --> cer2p_24_r + h2o_r + nadp_r
CERS2p26er: cer1_26_r + h_r + nadph_r + o2_r --> cer2p_26_r + h2o_r + nadp_r
CERS324er: cer2_24_r + h_r + nadph_r + o2_r --> cer3_24_r + h2o_r + nadp_r
CERS326er: cer2_26_r + h_r + nadph_r + o2_r --> cer3_26_r + h2o_r + nadp_r
DBTSm: atp_m + co2_m + dann_m <=> adp_m + dtbt_m + 3.0 h_m + pi_m
DHAOX_c: dhdascb_c + 2.0 gthrd_c --> ascb__L_c + gthox_c + h_c
DNTPPA: ahdt_c + h2o_c --> 